### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [1]:
%matplotlib qt5
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
import imp
import qsweepy.save_pkl

## Параметры кубитов

In [ ]:
db = database.database()
sweeper = sweep_extras.sweeper(db)

In [405]:
#Параметры кубитов.all keys should be str
Qubits = {'1':{'r':{'Fr': 7.278e9, 'P': -50},
               'q':{'00-1-01': 4.246158e9, 
                    '00-2-02': 4.12454e9,
                    '00-2-11': 4.09916e9,
                    '01-1-02': 4.0030e9, 
                    '00-1-10': 3.94780e9,
                    '00-2-20': 3.82293e9,
                    '10-1-20': 3.6990e9}},
        '2':{'r':{'Fr': 7.3388e9, 'P':-45}, #7.332e9,
               'q':{'00-1-01': 4.2835e9,#4.250412e9,#4.24947e9,
                    '00-2-02': 4.162e9,#4.12927e9,#4.129e9,  
                    '00-2-11': 4.1338e9,#4.1384e9 for 0.25,#4.1338e9 for 0.2,#4.1355e9,#4.127e9,#4.09495e9,
                    '00-1-10': 3.9633e9,#3.93139e9,#3.9316e9,
                    #'00-2-20': 3.8385e9,#3.7135e9,#3.80671e9,#3.80683e9,
                    #'10-1-20': 3.713e9
                   }}}#3.6762e9}}}                         
qubit_params=Qubits
qubits = Qubits
sample_id = '2'
sample_name = '3D_2'
exdir_db_inst = exdir_db.exdir_db(db, sample_name=sample_name)
#it should be dumped too
#qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
                                                                 #qjson.dump('qubits','qubit_params',qubits_dump)

In [364]:
qubits = {'1':{'r':{'Fr': 7.278e9, 'P': -3},
               'q1': 4.246158e9,
               'q2': 3.94780e9},
          '2':{'r':{'Fr': 7.332e9, 'P':-10}, 
               'q1': 4.250412e9,
               'q2': 3.93139e9}}        

## измерительные устройства

In [3]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
lo1._visainstrument.timeout = 5000

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [4]:
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)
awg1.get_clock = lambda: 1e9
awg2.get_clock = lambda: 1e9

In [5]:
adc_device = TSW14J56_evm()
adc_device.timeout = 10
adc = TSW14J56_evm_reducer(adc_device)
adc.output_raw = True
adc.last_cov = False
adc.avg_cov = False
adc.resultnumber = False

adc_device.set_trig_src_period(100*125) # 10 kHz period rate
adc_device.set_trig_src_width(2) # 80 ns trigger length



ADS54J40 already programmed


## Спектроскопия

In [6]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg 
#for channel in range(0,4):
#awg._visainstrument.write('AWGC:DOUT1 OFF')
#awg._visainstrument.write('AWGC:DOUT2 OFF')
#awg._visainstrument.write('AWGC:DOUT3 OFF')
#awg._visainstrument.write('AWGC:DOUT4 OFF')
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg2.trigger_behaviours = [0,4,4,4]
awg2.set_trigger_mode(2)
awg1.set_trigger_mode(2)
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
awg1.stop()
awg2.stop()
for channel in range(0,4):
    awg2.set_amplitude(0.05, channel=channel)
    awg2.set_offset(0.7,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*99500, channel=channel)
for channel in range(0,4):
    awg1.set_amplitude(0.03, channel=channel)
    awg1.set_offset(0.7,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*99500, channel=channel)
awg_tek.stop()
for channel in range(1,5):
    awg_tek.set_output(0, channel=channel)
    awg_tek.set_amplitude(1.0, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*99500, channel=channel)
awg2.set_marker(length=200, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
#     

# Задаём постояннmtые смещения на каналы 1 и 3
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [1]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-45)#(-45) # 
pna.set_nop(501)
pna.set_bandwidth(100)
pna.set_xlim(7.328e9, 7.35e9)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()
pna.pre_sweep()
plt.figure()
plt.plot(pna.get_points()['S-parameter'][0][1],np.log10(np.abs(pna.measure()['S-parameter']))*10)
pna.post_sweep()

NameError: name 'lo1' is not defined

In [1033]:
pna.get_power()

-80.0

In [10]:
sample_id = 2

# Однотоновая спектроскопия резонатора в зависимости от мощности


In [64]:
lo1.set_status(0)
pna.set_status(1)
#pna.set_power(-35) #
powers = np.linspace(-10, -60, 61)
pna.set_nop(101)
pna.set_xlim(7.33e9, 7.35e9)
pna.set_bandwidth(30)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()
meas = sweeper.sweep(pna, (powers, pna.set_power, 'Power', 'dBm'), 
                     sample_name = sample_id,
                     measurement_type = 'resonator_spectroscopy_powerscan',
                     metadata = {})

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}
load_exdir: file open time:  0.0019996166229248047
load_exdir: file open time:  0.000997781753540039
load_exdir: file open time:  0.001996755599975586
load time:  0.04197883605957031
pre_trace_time:  0.00099945068359375
trace 0 time:  0.024988174438476562
trace 1 time:  0.047975778579711914
trace 2 time:  0.07296490669250488
trace 3 time:  0.0969533920288086


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\memmap.py:335: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



## построение графиков задержки

In [69]:
delay_median = np.median(np.gradient(np.angle(meas['S-parameter'][2]), axis=1))
phase_compensator = np.exp(-1j*delay_median*np.arange(meas['S-parameter'][2].shape[1]))
meas_delay_compensated = meas['S-parameter'][2]*phase_compensator
meas_phase_median = np.median(np.angle(meas['S-parameter'][2]), axis=1)
#plt.figure('Однотоновая спектроскопия резонатора в зависимости от мощности')
meas_compensated = {'S-parameter':[i for i in meas['S-parameter']]}
meas_compensated['S-parameter'][2] = meas['S-parameter'][2]*phase_compensator

frequencies = meas_compensated['S-parameter'][1][1]/1e9
ang_freq_step = 2*np.pi*(frequencies[1]-frequencies[0])

#plotting.plot_measurement(meas_compensated)
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора в зависимости от мощности, образец #{}'.format(qubit_id))
plt.pcolormesh(meas_compensated['S-parameter'][1][0], 
               meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0]/ang_freq_step, cmap='RdBu')
plt.colorbar()
#plt.close()
plt.xlabel('Мощность, дБ')
plt.ylabel('Частота, ГГц')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy.pdf')


plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (-10 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,21]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

In [38]:
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (0 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,17]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

In [29]:
Qubits[sample_id]['r']['P']

-10

In [22]:
lo1.set_status(0)

True

# Двухтоновая спектроскопия (определение области частот, отвечающих кубитам)

In [273]:
ex_freqs = np.arange(4e9, 4.3e9, 1e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
pna.set_centerfreq(Qubits[sample_id]['r']['Fr'])
pna.set_frequency(Qubits[sample_id]['r']['Fr'])
pna.set_bandwidth(100)
lo1.set_power(-20)
powers = np.linspace(10, 20, 20)
result = sweeper.sweep(pna, #(powers, lo1.set_power, 'Second tone power', 'dBm'),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'GHz'), 
                       measurement_type = 'two_tone_spectroscopy_1D',
                       sample_name = '3D_' + str(sample_id),
                       metadata = {'power': lo1.get_power()})
lo1.set_status(0)

{}
load_exdir: file open time:  0.001978158950805664
load_exdir: file open time:  0.001027822494506836
load_exdir: file open time:  0.0010004043579101562
load time:  0.048950910568237305
pre_trace_time:  0.00099945068359375
trace 0 time:  0.02400946617126465
trace 1 time:  0.04397845268249512
trace 2 time:  0.06296944618225098
trace 3 time:  0.08297133445739746


True

# Двухтоновая спектроскопия при различных мощностях второго (возбуждающего) тона

In [1038]:
sample_id = '2'
ex_freqs = np.arange(4.10e9, 4.20e9, 0.1e6)#3.64e9, 4.4e9, 0.5e6)
ex_powers = np.arange(-20, 0, 1)
pna.set_centerfreq(Qubits[sample_id]['r']['Fr'])
pna.set_frequency(Qubits[sample_id]['r']['Fr'])
pna.set_nop(1)
pna.set_power(-75)
pna.set_bandwidth(20)
lo1.set_status(1)
result = sweeper.sweep(pna, 
                       (ex_powers, lo1.set_power, 'Power', 'dBm'), 
                       (ex_freqs, lo1.set_frequency, '2nd tone frequency', 'GHz'), 
                       measurement_type = 'two_tone_spectroscopy',
                       sample_name = '3D_' + str(sample_id),
                       comment = 'with_ampl',
                       metadata = {'VNA_power': pna.get_power()})
lo1.set_status(0)

{}
load_exdir: file open time:  0.0009984970092773438
load_exdir: file open time:  0.0009980201721191406
load_exdir: file open time:  0.0019998550415039062
load time:  0.044983863830566406
pre_trace_time:  0.0
trace 0 time:  0.030981063842773438
trace 1 time:  0.05896735191345215
trace 2 time:  0.08893489837646484
trace 3 time:  0.11994123458862305


True

# Точное наблюдение линии 00->11

In [74]:
ex_freqs = np.arange(4.075e9, 4.15e9, 0.1e6)
#ex_freqs = np.arange(4.08e9, 4.12e9,0.25e6)
ex_powers = np.arange(-20, 10, 1)

pna.set_xlim(Qubits[qubit_id]['r']['Fr'], Qubits[qubit_id]['r']['Fr'])
pna.set_nop(1)
pna.set_bandwidth(3)
if sample_id=='2':
    lo1.set_power(5)
    pna.set_power(Qubits[sample_id]['r']['P']-15)
if sample_id=='1':
    lo1.set_power(-1.5)
    pna.set_power(Qubits[sample_id]['r']['P']-15)
lo1.set_status(1)
sweeper.sweep(pna, 
              (ex_freqs, lo1.set_frequency, '2nd tone frequency', 'GHz'), 
              measurement_type = 'two_tone_spectroscopy_one_power',
              sample_name = '3D_' + str(sample_id),
              metadata = {'VNA_power': pna.get_power()})
lo1.set_status(0)

A Jupyter Widget

Started at:  Aug 29 2018 17:01:34
Exiting sweep main thread #{} via return 1564

Elapsed time: 0 h 4 m 4.22 s


True

# Измерить зависимость отражённого от резонатора сигнала от амплитуды считывающего импульса. Определить амплитуду насыщения, выставить амплитуду ниже уровня насыщения.

## Импульсный режим измерений

In [406]:
import imp
from qsweepy import pulses as pulses

imp.reload(awg_digital)


imp.reload(awg_iq_multi)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(tomography)
imp.reload(sweep)
imp.reload(sweep_extras)
imp.reload(pulses)
imp.reload(awg_channel)
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм

lo_ex_pow = 13
#lo_ex_pow = -20
lo_ex.set_status(1)
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

marker_length = 100
#readout_trigger_delay = 185
readout_trigger_delay = 0
trg_length = 10e-9

ex_clock = 1000e6
#ro_clock = 1250e6
ro_clock = 1000e6

rep_rate = 10e3 # частота повторений эксперимента
global_num_points = 100000-500

awg1.stop()
awg2.stop()
awg_tek.stop()

awg1.check_cached=True
awg2.check_cached=True
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg1.set_clock(ex_clock)
awg2.set_clock(ex_clock)
awg1.set_clock_output(True)
awg2.set_clock_output(True)
awg1.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg2.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg1.repetition_period = global_num_points
awg2.repetition_period = global_num_points

awg1.module.waveforms = [None]*4
awg1.module.AWGflush(nAWG=0) #убирает waveforms, которые были на этом канале
awg1.module.AWGflush(nAWG=1)
awg1.module.AWGflush(nAWG=2)
awg1.module.AWGflush(nAWG=3)
awg2.module.waveforms = [None]*4
awg2.module.AWGflush(nAWG=0)
awg2.module.AWGflush(nAWG=1)
awg2.module.AWGflush(nAWG=2)
awg2.module.AWGflush(nAWG=3)

# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar
awg2.set_trigger_mode(1) #external trigger mode
awg1.set_trigger_mode(1)

awg1.trigger_source_channels = [0, 0, 0, 0]
awg2.trigger_source_channels = [0, 0, 0, 0]#=0 for external trigger
#awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg1.trigger_delays = [1,1,1,1]
awg2.trigger_delays = [1,1,1,1]

awg1.trigger_behaviours = [4,4,4,4] #Trigger is active on the falling edge
awg2.trigger_behaviours = [4,4,4,4]

awg1.trigger_source_types = [6,6,6,6] # External HARDWARE trigger per cycle
awg2.trigger_source_types = [6,6,6,6]

for channel in range(0,4):
    #awg2.set_amplitude(0.4, channel=channel)
    awg2.set_offset(0.0,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*global_num_points, channel=channel)
for channel in range(0,4):
    #awg1.set_amplitude(1.0, channel=channel)
    awg1.set_offset(0.0,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*global_num_points, channel=channel)
    
awg1.set_amplitude(1.0, channel=0)
awg1.set_amplitude(1.0, channel=1)
awg1.set_amplitude(1.0, channel=2)
awg1.set_amplitude(1.0, channel=3)

##awg2.set_amplitude(1.0, channel=0)
#awg2.set_amplitude(1.0, channel=1)
#awg2.set_amplitude(1.0, channel=2)
#awg2.set_amplitude(1.0, channel=3)
    
print(awg1.module.clockResetPhase(3, 0))
print(awg2.module.clockResetPhase(3, 0))
    
awg2.set_marker(length=100, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
awg_tek.run()

awg_channels =dict()

0
0


In [407]:
awg_tek.stop()
awg_tek._visainstrument.write('AWGC:RMOD TRIG')
awg_tek._visainstrument.write('TRIG:WVAL LAST')
awg_tek._visainstrument.write('TRIG:IMP 1kohm')
awg_tek._visainstrument.write('TRIG:SLOP NEG')
awg_tek._visainstrument.write('TRIG:LEV 0.6')
awg_tek._visainstrument.write('SOUR1:ROSC:FREQ 10MHz')
awg_tek._visainstrument.write('SOUR1:ROSC:SOUR EXT')

#awg_tek.set_trigger_mode('CONT')
awg_tek.set_clock(1e9)
awg_tek.set_nop(ex_clock/rep_rate-500) # репрейт нужно задавать по=хорошему только на управляющей_t
awg_tek.check_cached = True

for channel in range(1,3):
    awg_tek.set_amplitude(1.0, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*global_num_points, channel=channel)
awg_tek.run()

awg_tek.set_digital([1]*90+[0]*(global_num_points+500-300), channel=1)
awg_tek.set_digital([1]*90+[0]*(global_num_points+500-300), channel=2)
for other_channels in [3,4,5,6,7,8]:
    awg_tek.set_digital([1]*1000+[0]*(global_num_points+500-1210), channel=other_channels)

awg_tek._visainstrument.write('SOUR1:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR2:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR3:DEL:ADJ 398 NS')
awg_tek._visainstrument.write('SOUR4:DEL:ADJ 398 NS')

#awg_tek._visainstrument.write('SOUR1:DEL:ADJ 398 NS')
#awg_tek._visainstrument.write('SOUR2:DEL:ADJ 398 NS')
#awg_tek._visainstrument.write('SOUR3:DEL:ADJ 398 NS')
#awg_tek._visainstrument.write('SOUR4:DEL:ADJ 398 NS')

awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 3)
ro_trg.mode = 'set_delay' #M3202A
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg

awg_tek.stop()
awg1.set_waveform([1]*200+[0]*(global_num_points-200), channel=0)
awg2.set_waveform([1]*200+[0]*(global_num_points-200), channel=0)

awg1.set_waveform([0]*200+[0]*(global_num_points-200), channel=0)
awg2.set_waveform([0]*200+[0]*(global_num_points-200), channel=0)
awg_tek.run()
awg1.run()
awg2.run()

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


In [408]:
awg1.get_amplitude(1)

1.0

## Настройки смесителей

In [409]:
#qjson.invalidate_calibrations()

In [410]:
#ro_if = 75e6
#lo_ro_freq = 7.338e9# for two-tone in order to have a rect non-modulated pulse
lo_ro_freq = 7.2e9 # for normal measurements
lo_freq = 4.35e9#4.1e9#3.9e9#35e9|
imp.reload(awg_iq_multi)

lo_ro.set_frequency(lo_ro_freq)
lo1.set_frequency(lo_freq)
readout_channels = {}

#ex_if = lo_freq-qubit_params[qubit_id]['q']['00-1-01']
#iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 1, 2, lo_ex) #M3202A
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C

#excitation channel 1
iq_ex = awg_iq_multi.awg_iq_multi(awg1, awg1, 2, 3, lo_ex) 
iq_ro = awg_iq_multi.awg_iq_multi(awg1, awg1, 0, 1, lo_ro)
iq_ex.calibration_switch_setter = lambda: print(rf_switch.do_set_switch(1, channel=1) if not rf_switch.do_get_switch(channel=1)==1 else None)
iq_ro.calibration_switch_setter = lambda: rf_switch.do_set_switch(4, channel=1) if not rf_switch.do_get_switch(channel=1)==4 else None

for tr, freq in qubits[sample_id]['q'].items():
    iq_ex.carriers['iq_ex_' + tr] = awg_iq_multi.carrier(iq_ex)
    iq_ex.carriers['iq_ex_' + tr].set_frequency(freq)
    awg_channels['iq_ex_' + tr] = iq_ex.carriers['iq_ex_' + tr]
    iq_ro.carriers['iq_ro_' + tr] = awg_iq_multi.carrier(iq_ro)
    iq_ro.carriers['iq_ro_' + tr].set_frequency(qubits[sample_id]['r']['Fr'])
    awg_channels['iq_ro_' + tr] = iq_ro.carriers['iq_ro_'+ tr]
    readout_channels['iq_ro_' + tr] = awg_channels['iq_ro_' + tr]
    #readout_channels = {'iq_ro_' + tr:awg_channels['iq_ro_'+ tr]}

pg = pulses.pulses(awg_channels)

print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

Excitation channel
c:\data/calibrations/iq-dc/ lo_freq-4.35e+09-mixer-default
c:\data/calibrations/iq-rf/ cname-default-frequency-4.284e+09-if--6.65e+07-sideband_id-     0
c:\data/calibrations/iq-rf/ cname-default-frequency-4.162e+09-if--1.88e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ cname-default-frequency-4.134e+09-if--2.162e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ cname-default-frequency-3.963e+09-if--3.867e+08-sideband_id-     0
max() arg is an empty sequence
None
-386700000.0 3963300000.0
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 5.00e-01-2.00e-01j	B: -45.84 G: -35.00, C:0.00
[-34.99556732 -73.02333832 -45.84038544]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 5.25e-01-2.00e-01j	B: -45.63 G: -34.81, C:0.00
[-34.80737686 -72.45511627 -45.6272583 ]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 5.00e-01-2.10e-01j	B: -45.59 G: -34.98, C:0.00
[-34.97620392 -72.95794678 -45.59067535]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 5.25e-01-1.90e-01j	B: -45.87 G: -34.83, C:0.00
[-34.828544

dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.51 G: -36.11, C:0.00
[-36.10544205 -74.18583679 -79.50544739]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.11, C:0.00
[-36.10576248 -74.19297791 -79.52613831]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.11, C:0.00
[-36.10590363 -74.17381287 -79.52733612]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.11, C:0.00
[-36.11003494 -74.14749908 -79.53484344]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.11, C:0.00
[-36.10517883 -74.13519287 -79.53057861]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.54 G: -36.11, C:0.00
[-36.11043549 -74.16304779 -79.53858948]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.11, C:0.00
[-36.106884   -74.17938232 -79.52765656]
dc: -1.11e-02-3.25e-02j	I:  4.00e-01	Q: 4.06e-01+1.16e-01j	B: -79.53 G: -36.10, C:0.00
[-36.10105896 -74

In [411]:
iq_ro.carriers

{'iq_ro_00-1-01': <qsweepy.awg_iq_multi.carrier at 0x218a2140860>,
 'iq_ro_00-2-02': <qsweepy.awg_iq_multi.carrier at 0x218a216d208>,
 'iq_ro_00-2-11': <qsweepy.awg_iq_multi.carrier at 0x218a216de48>,
 'iq_ro_00-1-10': <qsweepy.awg_iq_multi.carrier at 0x218a216d400>}

In [412]:
lo1.set_frequency(lo_freq)

True

In [413]:
iq_ex.dc_cname()

'lo_freq-4.35e+09-mixer-default'

In [161]:
iq_ex.get_dc_calibration()

c:\data/calibrations/iq-dc/ lo_freq-4.1e+09-mixer-default


OrderedDict([('dc', (-0.014344181698019692-0.04044454510101335j))])

In [162]:
iq_ex.get_rf_calibration(iq_ex.carriers['iq_ex_00-1-10'])

c:\data/calibrations/iq-rf/ cname-default-frequency-3.964e+09-if--1.36e+08-sideband_id-     0


OrderedDict([('I', 0.5),
             ('Q', (0.3954775261070894+0.10737653635971181j)),
             ('if', -136000000.0),
             ('lo_freq', 4100000000.0),
             ('num_sidebands', 9),
             ('score', -8.124999489903011)])

In [163]:
iq_ex.carriers['iq_ex_00-1-10'].get_if()

-136000000.0

## задержка считывания

In [13]:
#adc_device.system_reset()
#adc_device.sync_req()

In [414]:
trigger_readout_seq = [pg.p('ro_trg', 20e-9, pg.rect, 1)]
adc.set_nums(50)
adc.set_nop(65536)
#adc.set_posttrigger(adc.get_nop()-4096)
if hasattr(adc, 'set_posttrigger'):
    adc.set_posttrigger(adc.get_nop()-32)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
delay_calibration = modem.calibrate_delay()

readout_trigger_delay = delay_calibration[awg_channels['iq_ro_00-1-01']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock())-readout_trigger_delay*adc.get_clock()) #M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles-10)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro_00-1-01']]
## #readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem_calibration_result = modem.calibrate_dc(amplitude=0.5)

-7.37e-07 -737.0
-9e-09 -9.0
(49750, 2) (49750,)
(49750, 2) (49750,)
(49750, 2) (49750,)
(49750, 2) (49750,)


In [415]:
demod_q1 = modem_calibration_result['iq_ro_00-1-01']['feature'][:2048]/np.max(np.abs(modem_calibration_result['iq_ro_00-1-01']['feature']))
#demod_q2 = modem_calibration_result['iq_ro_00-2-02']['feature'][:2048]/np.max(np.abs(modem_calibration_result['iq_ro_00-2-02']['feature']))
#demod_q3 = modem_calibration_result['iq_ro_00-1-10']['feature'][:2048]/np.max(np.abs(modem_calibration_result['iq_ro_00-1-10']['feature']))
#3demod_q3 = calibration_result['iq_ro_q2']['feature'][:2048]/np.max(np.abs(calibration_result['iq_ro_q2']['feature']))
demod_q1 = np.asarray(2**15*demod_q1, dtype=complex)
#demod_q2 = np.asarray(2**15*demod_q2, dtype=complex)
#demod_q3 = np.asarray(2**15*demod_q3, dtype=complex)
demod_q1_real_int = np.asarray(np.real(demod_q1), dtype=np.int16)
demod_q1_imag_int = np.asarray(np.imag(demod_q1), dtype=np.int16)
#demod_q2_real_int = np.asarray(np.real(demod_q2), dtype=np.int16)
#demod_q2_imag_int = np.asarray(np.imag(demod_q2), dtype=np.int16)
#demod_q3_real_int = np.asarray(np.real(demod_q3), dtype=np.int16)
#demod_q3_imag_int = np.asarray(np.imag(demod_q3), dtype=np.int16)
adc_device.set_ram_data([demod_q1_real_int.tolist(),    (demod_q1_imag_int).tolist()], 0)
adc_device.set_ram_data([(demod_q1_imag_int).tolist(),  (-demod_q1_real_int).tolist()], 1)
#adc_device.set_ram_data([demod_q2_real_int.tolist(),    (demod_q2_imag_int).tolist()],  2)
#adc_device.set_ram_data([(demod_q2_imag_int).tolist(),  (-demod_q2_real_int).tolist()], 3)
#adc_device.set_ram_data([demod_q3_real_int.tolist(),    (demod_q3_imag_int).tolist()],  2)
#adc_device.set_ram_data([(demod_q3_imag_int).tolist(),  (-demod_q3_real_int).tolist()], 3)

## усреднённое считывание

In [416]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21_r{}'.format(transition): 
                       data_reduce.mean_reducer(modem, 'iq_ro_{}'.format(transition), axis=0) for transition in qubits[sample_id]['q'].keys()}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

In [417]:
###TSWWWWW!!~!!

adc_reducer = TSW14J56_evm_reducer(adc_device)
adc_reducer.output_raw = False
adc_reducer.last_cov = False
adc_reducer.avg_cov = True
adc_reducer.resultnumber = False
#adc_reducers = data_reduce.data_reduce(modem) 
#adc_reducers.filters = {'S21_r':data_reduce.mean_reducer(modem, 'iq_ro', axis=0)}
#adc_reducers.extra_opts['scatter'] = True
#adc_reducers.extra_opts['realimag'] = True

adc_reducers = {_qubit_id: data_reduce.data_reduce(modem) for  _qubit_id in qubits[sample_id]['q'].keys()}
for _qubit_id in qubits[sample_id]['q'].keys():
    adc_reducers[_qubit_id].filters = {'S21_r{}'.format(_qubit_id): 
                                       data_reduce.mean_reducer(modem, 'iq_ro_{}'.format(_qubit_id), axis=0)}
    adc_reducers[_qubit_id].extra_opts['scatter'] = True
    adc_reducers[_qubit_id].extra_opts['realimag'] = True
        
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_{}'.format(_qubit_id), axis=0) for _qubit_id in qubits[sample_id]['q'].keys()}

## длительность считывания

In [418]:
def set_adc_nop(ro_adc_length):
    #adc.stop() # Spectrum
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    #print (nop)
    adc.set_nop(nop)
    #adc.set_post_trigger(adc.get_nop()-64)
    
# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
    
    
#ro_adc_length = 0.15e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
#ro_amplitude =0.16
#ro_dac_length = 0.08e-6
ro_adc_length = 0.3e-6
ro_dac_length = 0.2e-6
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
adc.set_nums(100)
set_adc_nop(ro_adc_length)
adc.set_nums(10000)

## измерение среднего прошедшего импульса

In [419]:
# это не нужно нажимать

In [18]:
mean_sample = data_reduce.data_reduce(adc)
mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

#adc_reducer_old = data_reduce.data_reduce(adc)
#adc_reducer_old.filters['Std Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage std', 0)
mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro'].get_if())
mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro'].get_if())
#adc_downsampler = data_reduce.data_reduce(adc)
#adc_downsampler.filters['Voltage'] = data_reduce.downsample_reducer(adc, 'Voltage', 0, iq_ro.get_if(), 4)
# # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
mean_sample.extra_opts['scatter'] = True

awg.check_cached = True

NameError: name 'awg' is not defined

## Прохождение считывающих импульсов разной амплитуды

In [169]:
imp.reload(sweep)
imp.reload(plotting)
for _qubit_id in qubits[sample_id]['q'].keys():
    #for coil_channel_id, channel_voltage in qubits[_qubit_id]['q']['P']['Fmin_coil_currents'].items():
        #dac.set_voltage(channel_voltage, channel=nndac_channel_id)
    #    coil_device.set_offset(channel_voltage, channel=coil_channel_id)
    mean_sample = data_reduce.data_reduce(adc)
    #print (dir(mean_sample))
    #del mean_sample.measure_deferred_result
    mean_sample.filters['Mean_Voltage_AC'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

    mean_sample.filters['S21p'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro_{}'.format(_qubit_id)].get_if())
    mean_sample.filters['S21m'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro_{}'.format(_qubit_id)].get_if())
    # # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
    # # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
    mean_sample.extra_opts['scatter'] = True

    #awg.check_cached = True

    mean_sample.filters['Std_Voltage_AC'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
    def set_pulse_amplitude(x):
        #awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
        pg.set_seq(trigger_readout_seq+[pg.p('iq_ro_{}'.format(_qubit_id), ro_dac_length, pg.rect, x)])
        #awg_tek.run()
    measurement = sweeper.sweep(mean_sample, 
                            (np.linspace(0.0, 0.5, 21), set_pulse_amplitude, 'dac_amp'), 
                            measurement_type='readout_passthrough',
                            sample_name = '3D_' + str(sample_id))
    del mean_sample.filters['Std_Voltage_AC']

{}
Waiting to join deferred threads:
load_exdir: file open time:  0.0010006427764892578
load_exdir: file open time:  0.0019996166229248047
load_exdir: file open time:  0.0009999275207519531
load time:  0.13892412185668945
pre_trace_time:  0.000989675521850586
trace 0 time:  0.028008460998535156
trace 1 time:  0.05299544334411621
trace 2 time:  0.07896113395690918
trace 3 time:  0.10494875907897949
trace 4 time:  0.1169428825378418
trace 5 time:  0.128936767578125
trace 6 time:  0.14093613624572754
trace 7 time:  0.15392541885375977
trace 8 time:  0.16492033004760742
trace 9 time:  0.17791390419006348
trace 10 time:  0.18990850448608398
trace 11 time:  0.20290160179138184
trace 12 time:  0.2278916835784912
trace 13 time:  0.2538795471191406
trace 14 time:  0.27986574172973633
trace 15 time:  0.30785179138183594
{}


KeyboardInterrupt: 

In [179]:
imp.reload(sweep)
imp.reload(plotting)
mean_sample = data_reduce.data_reduce(adc)
mean_sample.filters['Mean_Voltage_AC'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
mean_sample.filters['S21p'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro'].get_if())
mean_sample.filters['S21m'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro'].get_if())
mean_sample.extra_opts['scatter'] = True
mean_sample.filters['Std_Voltage_AC'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
def set_pulse_amplitude(x):
    #awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq(trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    #awg_tek.run()
measurement = sweeper.sweep(mean_sample, 
                            (np.linspace(0.0, 1., 21), set_pulse_amplitude, 'dac amp'), 
                            measurement_type='readout_passthrough',
                            sample_name = '3D_' + str(sample_id))
del mean_sample.filters['Std_Voltage_AC']

KeyError: 'iq_ro'

In [420]:
#ставим ручками
ro_amplitude = 0.1
readout_pulse = [ pg.pmulti(ro_dac_length, ('iq_ro_00-1-01', pg.rect, ro_amplitude))] 

In [33]:
#ro_parameters = {"adc_len":ro_adc_length,'dac_len': ro_dac_length,'amp': ro_amplitude,'adc nums':adc.nums,
#                'trg_length':trg_length}
#qjson.dump("setups","readout",ro_parameters)

In [34]:
#ro_amplitude = 1.0
#o_sequence = trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

## Двухтонка импульсами

In [421]:
imp.reload(qsweepy)
from qsweepy import*

In [422]:
import qsweepy.qjson as pqjson
imp.reload(two_level)
imp.reload(sweep_extras)
imp.reload(sweep)
import qsweepy.two_level as dyn

In [423]:
imp.reload(dyn)
imp.reload(qjson)
imp.reload(data_reduce)
imp.reload(plotting)
imp.reload(fitting)
imp.reload(sweep)
imp.reload(pulses)
imp.reload(diff_readout)

<module 'qsweepy.diff_readout' from 'C:\\qtlab_replacement\\qsweepy\\diff_readout.py'>

In [424]:
imp.reload(awg_iq_multi)

<module 'qsweepy.awg_iq_multi' from 'C:\\qtlab_replacement\\qsweepy\\awg_iq_multi.py'>

In [23]:
imp.reload(two_level)
qubit_dynamics = {}
fitters = {}
ex_channels = ['00-1-01', 
               '00-1-10', 
               #'00-2-11', 
               #'00-2-02',                
               #'iq_ex_00-2-20', 
              ]

import qsweepy.two_level as dyn
imp.reload(pulses)
imp.reload(two_level)
imp.reload(dyn)
for ex_channel in ex_channels:
    qubit_dynamics[ex_channel] = {}
    ro_meas_name = 'avg_cov0'
    fitters[ex_channel] = lambda x,y,ro_meas_name=ro_meas_name: fitting.fit1d(x,y,ro_meas_name)
    qubit_dynamics[ex_channel] = dyn.quantum_two_level_dynamics(pg,
                                               adc_reducer, #adc_reducers_chan[ex_channel],
                                               ex_channel='iq_ex_' +ex_channel,
                                               ro_channel='iq_ro_' + ex_channel,
                                               ro_sequence=(trigger_readout_seq + readout_pulse),
                                               ex_amplitude=0.5,
                                               fitter=fitters[ex_channel], 
                                               shuffle=False, sample_name = '3D_' + str(sample_id),
                                               readout_measurement_name=ro_meas_name,  
                                               qubit_id = sample_id + ': ' + ex_channel, 
                                               comment = ex_channel, db=db)

NameError: name 'readout_pulse' is not defined

In [64]:
qubit_dynamics['00-1-10'].ex_channels[0]

'iq_ex_00-1-10'

In [487]:
pg.channels['iq_ex_00-1-01'].get_frequency()

4283500000.0

In [480]:
pg.channels

{'ro_trg': <qsweepy.awg_digital.awg_digital at 0x17b359e07b8>,
 'iq_ex_00-1-01': <qsweepy.awg_iq_multi.carrier at 0x17b2bc8ae80>,
 'iq_ro_00-1-01': <qsweepy.awg_iq_multi.carrier at 0x17b2bbfb278>,
 'iq_ex_00-1-10': <qsweepy.awg_iq_multi.carrier at 0x17b2bbfbba8>,
 'iq_ro_00-1-10': <qsweepy.awg_iq_multi.carrier at 0x17b2bbfb550>}

In [481]:
#проверить _if: она выставляется?
#калибровку подбирает вроде бы ту

In [484]:
iq_ex.carriers['iq_ex_00-1-10'].ignore_calibration_drift = True

In [486]:
iq_ex.carriers['iq_ex_00-1-10'].ignore_calibration_drift

True

In [573]:
pna.set_power(-45)

True

In [619]:
qubits[sample_id]['q']['00-1-10'],

(3964000000.0,)

In [620]:
lo_freq

3900000000.0

In [621]:
pna.set_nop(1)
pna.set_centerfreq(Qubits[sample_id]['r']['Fr'])
pna.set_frequency(Qubits[sample_id]['r']['Fr'])
pna.set_bandwidth(100)
qubit_dynamics['00-1-10'].two_tone(pna, np.linspace(0.3, 1, 8), 
                                   np.arange(3.93e9, 4.02e9, 2e6), 
                                   lo_freq,#qubits[sample_id]['q']['00-1-10'], 
                                   1.835e+08, iq_ex.carriers['iq_ex_00-1-10'])
                                   #iq_ex.carriers['iq_ex_00-1-10'], lo_freq)#iq_ex.carriers['iq_ex_00-1-10'].get_frequency())

{}
I-ve changed ampl
My IF is  30000000.0
My IF is  32000000.0
My IF is  34000000.0
My IF is  36000000.0
My IF is  38000000.0
My IF is  40000000.0
My IF is  42000000.0
My IF is  44000000.0
My IF is  46000000.0
My IF is  48000000.0
My IF is  50000000.0
My IF is  52000000.0
My IF is  54000000.0
My IF is  56000000.0
My IF is  58000000.0
My IF is  60000000.0
My IF is  62000000.0
My IF is  64000000.0
My IF is  66000000.0
My IF is  68000000.0
My IF is  70000000.0
My IF is  72000000.0
My IF is  74000000.0
My IF is  76000000.0
My IF is  78000000.0
My IF is  80000000.0
My IF is  82000000.0
My IF is  84000000.0
My IF is  86000000.0
My IF is  88000000.0
My IF is  90000000.0
My IF is  92000000.0
My IF is  94000000.0
My IF is  96000000.0
My IF is  98000000.0
My IF is  100000000.0
My IF is  102000000.0
My IF is  104000000.0
My IF is  106000000.0
My IF is  108000000.0
My IF is  110000000.0
My IF is  112000000.0
My IF is  114000000.0
My IF is  116000000.0
My IF is  118000000.0
I-ve changed ampl
My IF 

In [273]:
awg2.get_amplitude(3)

1.0

In [ ]:
iq_ex.carriers['iq_ex_00-1-10'].get_frequency()

In [210]:
qubits[sample_id]['q']['00-1-10']

3964000000.0

## Раби+Рамзи

In [425]:
ex_channels = ['00-1-01', 
               '00-2-02',  
               #'00-1-10', 
               #'00-2-11', 
               #'00-2-02',                
               #'iq_ex_00-2-20', 
              ]

ex_amplitudes = {'iq_ex_00-1-01':1, 
                 'iq_ex_00-1-10':1, 
                 'iq_ex_00-2-11':1, 
                 #'iq_ex_00-2-02':1,                
                 #'iq_ex_00-2-20':1, 
                }
Rabi_lengths = {'00-1-01':np.arange(0., 100e-9, 2e-9), 
                '00-1-10':np.arange(0., 200e-9, 2e-9),
                '00-2-11':np.arange(0., 1500e-9, 20e-9), 
                '00-2-02':np.arange(0., 200e-9, 5e-9), 
                #'iq_ex_00-2-20':np.arange(0., 500e-9, 10e-9),
                }
decay = {'iq_ex_00-1-01':np.arange(0., 5e-6, 50e-9), 
                      'iq_ex_00-1-10':np.arange(0., 5e-6, 50e-9), 
                      'iq_ex_00-2-11':np.arange(0., 10e-6, 100e-9), 
                      #'iq_ex_00-2-02':np.arange(0., 1e-6, 20e-9), 
                      #'iq_ex_00-2-20':np.arange(0., 1e-6, 20e-9),
                     }
#ro_par = {'qubit_id':qubit_id,'ex_channels':ex_channels}
#fitter = lambda x,y: fitting.fit1d(x,y,'S21')
#ro_par['fitter']=fitter

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
decay = np.arange(0, 200e-9, 2e-9)
target_freq_offset = -20e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -2e6
delaysT1 = np.arange(0, 15e-6,0.05e-6)

In [426]:
Rabi_results = {ex_channel:{} for ex_channel in ex_channels}
Ramsey_results = {ex_channel:{} for ex_channel in ex_channels}
decay_results = {ex_channel:{} for ex_channel in ex_channels}

In [427]:
import qsweepy.qjson as pqjson
imp.reload(two_level)
imp.reload(sweep_extras)
imp.reload(sweep)
import qsweepy.two_level as dyn

In [428]:
qubit_dynamics = {}

In [391]:
target_offset = 20e6
for exc_ampl in [0.2, 0.1]:
    for ex_channel in ex_channels:
        #old_if = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].ex_channels[0]
        Rabi_results[ex_channel+ '_'+ str(exc_ampl)] = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].Rabi(np.arange(0., 80e-9, 1e-9))
        #Ramsey_results[ex_channel+ '_'+ str(exc_ampl)] = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].Ramsey(np.linspace(0,200e-9,90), target_offset)
        #decay_results[ex_channel] = decay_dynamics[ex_channel].Rabi(decay_lengths)

KeyError: '00-1-01_0.2'

In [429]:
imp.reload(two_level)
fitters = {}
ex_channels = ['00-1-01', 
               '00-1-10', 
               #'00-2-11', 
               #'00-2-02',                
               #'iq_ex_00-2-20', 
              ]

import qsweepy.two_level as dyn
import qsweepy.pulses as pulses
imp.reload(pulses)
imp.reload(two_level)
imp.reload(dyn)
#for exc_ampl in [0.1, 0.2, 0.3]:
for ex_channel in ex_channels:
        qubit_dynamics[ex_channel + '_'+ str(exc_ampl)] = {}
        ro_meas_name = 'avg_cov1'
        fitters[ex_channel] = lambda x,y,ro_meas_name=ro_meas_name: fitting.fit1d(x,y,ro_meas_name)
        qubit_dynamics[ex_channel] = dyn.quantum_two_level_dynamics(pg,
                                                   adc_reducer, #adc_reducers_chan[ex_channel],
                                                   ex_channel='iq_ex_' +ex_channel,
                                                   ro_channel='iq_ro_00-1-01',# + ex_channel,
                                                   ro_sequence=(trigger_readout_seq + readout_pulse),
                                                   ex_amplitude=0.2,
                                                   fitter=fitters[ex_channel], 
                                                   shuffle=False, sample_name = '3D_' + str(sample_id),
                                                   readout_measurement_name=ro_meas_name,  
                                                   qubit_id = sample_id + ': ' + ex_channel, 
                                                   comment =ex_channel, exdir_db=exdir_db_inst)

c:\data/calibrations/setups/ readout
Failed loading readout calibration: expected str, bytes or os.PathLike object, not NoneType
c:\data/calibrations/two-level-rabi-rect/ c-iq_ex_00-1-01-f-4.2835e+09-amp-r0.2-i000-1-01
c:\data/calibrations/two-level-ramsey/ c-iq_ex_00-1-01-f-4.2835e+09-amp-r0.2-i000-1-01
c:\data/calibrations/setups/ readout
Failed loading readout calibration: expected str, bytes or os.PathLike object, not NoneType
c:\data/calibrations/two-level-rabi-rect/ c-iq_ex_00-1-10-f-3.9633e+09-amp-r0.2-i000-1-10
Failed loading rabi frequency calibration: max() arg is an empty sequence
c:\data/calibrations/two-level-ramsey/ c-iq_ex_00-1-10-f-3.9633e+09-amp-r0.2-i000-1-10
Failed loading ramsey calibration: max() arg is an empty sequence


In [430]:
Rabi_results = {ex_channel+ '_'+ str(exc_ampl):{} for ex_channel in ex_channels}
Ramsey_results = {ex_channel+ '_'+ str(exc_ampl):{} for ex_channel in ex_channels}

In [431]:
Rabi = {ex_channel: {} for ex_channel in ex_channels}
for ex_channel in ex_channels:
        #old_if = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].ex_channels[0]
        Rabi_results[ex_channel] = qubit_dynamics[ex_channel].Rabi(np.arange(0., 80e-9, 1e-9))

{}
load_exdir: file open time:  0.0009953975677490234
load_exdir: file open time:  0.0019958019256591797
load_exdir: file open time:  0.0019996166229248047
load time:  0.12894201278686523
pre_trace_time:  0.000997781753540039
trace 0 time:  0.014991283416748047
trace 1 time:  0.026984453201293945
trace 2 time:  0.03898000717163086
trace 3 time:  0.05096864700317383
fitted params:  {'phase': 6.663112792370968, 'freq': 74412999.78596194, 'decay': 4.25853104644524e-07, 'amplitudes': array([5.72702054e+11]), 'initial_points': array([-6.04303155e+12])}
1743
{}
load_exdir: file open time:  0.0009996891021728516
load_exdir: file open time:  0.0009996891021728516
load_exdir: file open time:  0.0009996891021728516
load time:  0.1209416389465332
pre_trace_time:  0.0010004043579101562
trace 0 time:  0.012993812561035156
trace 1 time:  0.02498793601989746
trace 2 time:  0.03498387336730957
trace 3 time:  0.04597759246826172
fitted params:  {'phase': 6.516787173085061, 'freq': 73231415.14119934, 'd

In [432]:
for ex_channel in ex_channels:
        #old_if = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].ex_channels[0]
        Ramsey_results[ex_channel] = qubit_dynamics[ex_channel].Ramsey(np.arange(0., 800e-9, 5e-9), target_offset)

{}
load_exdir: file open time:  0.0009992122650146484
load_exdir: file open time:  0.000997304916381836
load_exdir: file open time:  0.002000570297241211
load time:  0.1219491958618164
pre_trace_time:  0.0009965896606445312
trace 0 time:  0.012988805770874023
trace 1 time:  0.024984121322631836
trace 2 time:  0.034978389739990234
trace 3 time:  0.04597353935241699
fitted params:  {'phase': 3.0307239459280595, 'freq': 19898296.435133077, 'decay': -30.265166419046956, 'amplitudes': array([4.34279175e+11]), 'initial_points': array([-7.07405693e+12])}
{}
load_exdir: file open time:  0.0009965896606445312
load_exdir: file open time:  0.0009968280792236328
load_exdir: file open time:  0.0010030269622802734
load time:  0.1199483871459961
pre_trace_time:  0.0009970664978027344
trace 0 time:  0.01199030876159668
trace 1 time:  0.02398371696472168
trace 2 time:  0.034980058670043945
trace 3 time:  0.04497790336608887
fitted params:  {'phase': 2.8603425571251293, 'freq': 19962390.672353882, 'deca

In [442]:
qubit_dynamics[control].Rabi_rect_result

{'rabi_rect_initial_points': array([-5.94345754e+12]),
 'rabi_rect_phase': 6.516787173085061,
 'rabi_rect_amplitudes': array([1.3829702e+12]),
 'rabi_rect_freq': 73231415.14119934,
 'rabi_rect_decay': -4.090133181868861,
 'rabi_carriers': [3963300000.0],
 'rabi_ro_freq': 7338800000.0,
 'rabi_ex_amplitudes': (0.2,),
 'rabi_tail_length': 0.0,
 'qubit_id': '2: 00-1-10'}

In [453]:
for ex_channel in ex_channels:
        #old_if = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].ex_channels[0]
        old_comment = qubit_dynamics[ex_channel].comment
        qubit_dynamics[ex_channel].comment='ZZ-crosstalk'
        control = [c for c in ex_channels if c != ex_channel][0]
        pre_pulse_seq = [pg.p('iq_ex_'+control, 1/(2*qubit_dynamics[control].Rabi_rect_result['rabi_rect_freq']), pg.rect, qubit_dynamics[control].ex_amplitudes[0])]
        Ramsey_results[ex_channel] = qubit_dynamics[ex_channel].Ramsey(np.arange(0., 2000e-9, 20e-9), 5e6, pre_pulse_seq=pre_pulse_seq)
        qubit_dynamics[ex_channel].comment=''

{}
load_exdir: file open time:  0.000993967056274414
load_exdir: file open time:  0.0020008087158203125
load_exdir: file open time:  0.0009822845458984375
load time:  0.1259605884552002
pre_trace_time:  0.0010013580322265625
trace 0 time:  0.0139923095703125
trace 1 time:  0.024987459182739258
trace 2 time:  0.035980224609375
trace 3 time:  0.046976566314697266
fitted params:  {'phase': 2.672527714014002, 'freq': 4883601.3690336365, 'decay': 8.292578616493925e-07, 'amplitudes': array([8.8806866e+11]), 'initial_points': array([-1.04782129e+13])}
{}
load_exdir: file open time:  0.0009949207305908203
load_exdir: file open time:  0.0019834041595458984
load_exdir: file open time:  0.0009951591491699219
load time:  0.12694454193115234
pre_trace_time:  0.0010018348693847656
trace 0 time:  0.013990402221679688
trace 1 time:  0.02498626708984375
trace 2 time:  0.03698110580444336
trace 3 time:  0.04897427558898926
fitted params:  {'phase': 2.8584679140002502, 'freq': 4905173.555639634, 'decay':

In [454]:
for ex_channel in ex_channels:
        #old_if = qubit_dynamics[ex_channel+ '_'+ str(exc_ampl)].ex_channels[0]
        
        Ramsey_results[ex_channel] = qubit_dynamics[ex_channel].Ramsey(np.arange(0., 2000e-9, 20e-9), 5e6)

{}
load_exdir: file open time:  0.0009958744049072266
load_exdir: file open time:  0.0009970664978027344
load_exdir: file open time:  0.000993490219116211
load time:  0.12194561958312988
pre_trace_time:  0.0009992122650146484
trace 0 time:  0.012989044189453125
trace 1 time:  0.02398514747619629
trace 2 time:  0.034979820251464844
trace 3 time:  0.04597282409667969
fitted params:  {'phase': 3.0126054354102396, 'freq': 4872091.34238167, 'decay': 1.7196437137402434e-06, 'amplitudes': array([5.14305238e+11]), 'initial_points': array([-7.15895334e+12])}
{}
load_exdir: file open time:  0.0019903182983398438
load_exdir: file open time:  0.0019979476928710938
load_exdir: file open time:  0.0009984970092773438
load time:  0.12694931030273438
pre_trace_time:  0.0009965896606445312
trace 0 time:  0.01298666000366211
trace 1 time:  0.022982120513916016
trace 2 time:  0.03497815132141113
trace 3 time:  0.0459749698638916
fitted params:  {'phase': 2.882124372527652, 'freq': 4954517.223811241, 'deca

In [455]:
from qsweepy import gauss_hd_calibration
imp.reload(gauss_hd_calibration)
imp.reload(fitting)
qubit_dynamics['00-1-01'].exdir_db = exdir_db_inst
qubit_dynamics['00-1-10'].exdir_db = exdir_db_inst
drag_hds =         {'1':gauss_hd_calibration.gauss_hd_calibration(qubit_dynamics['00-1-01'], 
                                     sigma=4e-9, 
                                     length=20e-9,
                                     alpha=0.0, 
                                     anharmonicity=254e6),
                    '2':gauss_hd_calibration.gauss_hd_calibration(qubit_dynamics['00-1-10'], 
                                      sigma=4e-9, 
                                      length=20e-9,
                                      alpha=0.0, 
                                      anharmonicity=253e6),
                 }


In [456]:
iters = 1
for _iter in range(iters):
    for q_id, gauss_calib in drag_hds.items():
            gauss_calib.calibrate_amplitude(max_angle=np.pi, num_pulses_max=None)
            gauss_calib.save_calibration()

num_pulses_in_scan:  [ 2  4  8 16 32]
length:  2e-08
sigma:  4e-09
cutoff:  0.043936933623407476
erf_arg:  1.7677669529663684
erf result 0.9937903346742238
infinite length result:  1.0026513098524002e-08
finite length result 9.50304792105705e-09
length:  2e-08
sigma:  4e-09
cutoff:  0.043936933623407476
erf_arg:  1.7677669529663684
erf result 0.9937903346742238
infinite length result:  1.0026513098524002e-08
finite length result 9.50304792105705e-09
9.50304792105705e-09 4.443156393246919
Rabi freq: 74412999.78596194
2
target_angles: [0.         3.14159265 6.28318531]
amplitude_guesses_kv:  {0: 0.0, 4.443156393246919: 1.0}
range_mins: [0.35353163]
range_maxs: [1.06059489]
{}
load_exdir: file open time:  0.0010182857513427734
load_exdir: file open time:  0.0009984970092773438
load_exdir: file open time:  0.0009989738464355469
load time:  0.12392330169677734
pre_trace_time:  0.001020193099975586
trace 0 time:  0.012993335723876953
trace 1 time:  0.025966882705688477
trace 2 time:  0.03798

load_exdir: file open time:  0.001979827880859375
load_exdir: file open time:  0.0009992122650146484
load_exdir: file open time:  0.001001596450805664
load time:  0.11994767189025879
pre_trace_time:  0.0009779930114746094
trace 0 time:  0.013011693954467773
trace 1 time:  0.02400517463684082
trace 2 time:  0.03500056266784668
trace 3 time:  0.04599428176879883
{1793: 'fit_source'}
{}
load_exdir: file open time:  0.0019986629486083984
load_exdir: file open time:  0.0009996891021728516
load_exdir: file open time:  0.0019986629486083984
load time:  0.11694717407226562
pre_trace_time:  0.0009968280792236328
trace 0 time:  0.0139923095703125
trace 1 time:  0.02498769760131836
trace 2 time:  0.03598213195800781
trace 3 time:  0.04697704315185547
{1795: 'fit_source'}
{}
load_exdir: file open time:  0.002015352249145508
load_exdir: file open time:  0.0010020732879638672
load_exdir: file open time:  0.0009984970092773438
load time:  0.12294411659240723
pre_trace_time:  0.0009794235229492188
tra

{1829: 'fit_source'}
8
target_angles: [0.         0.78539816 1.57079633 2.35619449 3.14159265 3.92699082]
amplitude_guesses_kv:  {0: 0.0, 1.5707963267948966: 0.3592358434733467, 3.141592653589793: 0.7184716869466934}
range_mins: [0.08980896 0.26942688 0.4490448  0.62866273]
range_maxs: [0.26942688 0.4490448  0.62866273 0.80828065]
{}
load_exdir: file open time:  0.0019989013671875
load_exdir: file open time:  0.0019752979278564453
load_exdir: file open time:  0.0009799003601074219
load time:  0.1259622573852539
pre_trace_time:  0.0009987354278564453
trace 0 time:  0.013972043991088867
trace 1 time:  0.02498340606689453
trace 2 time:  0.036960601806640625
trace 3 time:  0.04797840118408203
{1831: 'fit_source'}
{}
load_exdir: file open time:  0.0019762516021728516
load_exdir: file open time:  0.0009882450103759766
load_exdir: file open time:  0.0019991397857666016
load time:  0.12594056129455566
pre_trace_time:  0.0009996891021728516
trace 0 time:  0.0139923095703125
trace 1 time:  0.024

{1863: 'fit_source'}
{}
load_exdir: file open time:  0.0019927024841308594
load_exdir: file open time:  0.0010170936584472656
load_exdir: file open time:  0.0010182857513427734
load time:  0.11992454528808594
pre_trace_time:  0.0
trace 0 time:  0.014014005661010742
trace 1 time:  0.026011228561401367
trace 2 time:  0.03700447082519531
trace 3 time:  0.04800271987915039
{1865: 'fit_source'}
{}
load_exdir: file open time:  0.002018451690673828
load_exdir: file open time:  0.0009946823120117188
load_exdir: file open time:  0.0009932518005371094
load time:  0.12294173240661621
pre_trace_time:  0.0
trace 0 time:  0.014974355697631836
trace 1 time:  0.028964757919311523
trace 2 time:  0.03995943069458008
trace 3 time:  0.052953243255615234
{1867: 'fit_source'}
{}
load_exdir: file open time:  0.0019931793212890625
load_exdir: file open time:  0.0009980201721191406
load_exdir: file open time:  0.0009958744049072266
load time:  0.11695003509521484
pre_trace_time:  0.0009999275207519531
trace 0 

In [457]:
iters = 1
for _iter in range(iters):
    for q_id, gauss_calib in drag_hds.items():
            gauss_calib.load_calibration()

c:\data/calibrations/gauss_amp/ c-iq_ex_00-1-01-f-4.2835e+09-amp-r0.2-i0ZZ-crosstalk-s-4T-20
c:\data/calibrations/gauss_amp/ c-iq_ex_00-1-10-f-3.9633e+09-amp-r0.2-i000-1-10-s-4T-20


In [152]:
pi_pulse1 = drag_hds['1'].get_pulse_seq(np.pi, 0)
pi_pulse2 = drag_hds['2'].get_pulse_seq(np.pi, 0)

In [211]:
pi_pulse1

[{'ro_trg': array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
  'iq_ex_00-1-01': array([0.        +0.j, 0.00687511+0.j, 0.01724203+0.j, 0.03170233+0.j,
         0.05026952+0.j, 0.07205051+0.j, 0.09510086+0.j, 0.11659135+0.j,
         0.13332768+0.j, 0.14251383+0.j, 0.14251383+0.j, 0.13332768+0.j,
         0.11659135+0.j, 0.09510086+0.j, 0.07205051+0.j, 0.05026952+0.j,
         0.03170233+0.j, 0.01724203+0.j, 0.00687511+0.j, 0.        +0.j]),
  'iq_ro_00-1-01': array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
  'iq_ex_00-2-02': array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
  'iq

In [29]:
ex_transition = '00-2-11'
ex_channel = 'iq_ex_' + ex_transition

In [30]:
ex_amplitude = 0.2

In [31]:
qubit_dynamics[ex_transition+ '_'+ str(ex_amplitude)].Rabi_2D(np.arange(4.1326e9, 4.1350e9, 0.1e6), np.arange(0., 1500e-9, 10e-9), iq_ex.carriers[ex_channel])

{}
load_exdir: file open time:  0.0020003318786621094
load_exdir: file open time:  0.001995086669921875
load_exdir: file open time:  0.002016782760620117
load time:  0.14095520973205566
pre_trace_time:  0.003997325897216797
trace 0 time:  0.02498340606689453
trace 1 time:  0.04697561264038086
trace 2 time:  0.06896305084228516
trace 3 time:  0.09195399284362793


In [146]:
qjson.dump(type='two-level-ramsey',name=qubit_dynamics['00-1-10'].build_calibration_filename(), params=qubit_dynamics['00-1-10'].Ramsey_result)

In [148]:
qjson.dump(type='two-level-rabi-rect',name = qubit_dynamics['00-1-01'].build_calibration_filename(), params=qubit_dynamics['00-1-01'].Rabi_rect_result)

In [437]:
for exc_ampl in [1.0, 0.8, 0.6]:
    ex_transition = '00-2-11'
    ex_channel = 'iq_ex_' + ex_transition
    qubit_dynamics[ex_transition+ '_'+ str(exc_ampl)].Rabi_2D(np.arange(4.125e9, 4.145e9, 0.1e6), np.arange(0., 1500e-9, 30e-9), iq_ex.carriers[ex_channel])

{}


KeyboardInterrupt: 

In [72]:
ex_transition = '00-2-11'
ex_channel = 'iq_ex_'+ ex_transition
ex_ampl = 1.0
amplitudes = np.linspace(0.0, 0.5, 51)
print(amplitudes)
frequencies = np.linspace(4.1e9, 4.2e9, 501)
qubit_dynamics[ex_transition+ '_'+ str(ex_ampl)].two_tone_length = 10e-6
qubit_dynamics[ex_transition+ '_'+ str(ex_ampl)].two_tone(amplitudes, frequencies, iq_ex.carriers[ex_channel], tail_length=0)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41
 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 ]
{}
load_exdir: file open time:  0.0019986629486083984
load_exdir: file open time:  0.0020165443420410156
load_exdir: file open time:  0.0019998550415039062
load time:  0.14593172073364258
pre_trace_time:  0.001976490020751953
trace 0 time:  0.025005578994750977
trace 1 time:  0.04700040817260742
trace 2 time:  0.06998467445373535
trace 3 time:  0.09197592735290527


In [ ]:
frequencies = np.linspace(4.1e9, 4.2e9, 1001)
qubit_dynamics[ex_transition+ '_'+ str(exc_ampl)].two_tone_length = 10e-6
qubit_dynamics[ex_transition+ '_'+ str(exc_ampl)].two_tone(amplitudes, frequencies, iq_ex.carriers[ex_channel], tail_length=0)

## Tomography

In [490]:
imp.reload(joined_readout)
readout = joined_readout.joined_readout(drag_hds)
readout.get_disp_shifts()
readout.states

{'00': {'avg_cov0': -8638209196591,
  'avg_cov1': -6134102225210,
  'avg_cov2': -8478771342343,
  'avg_cov3': -6429425378425},
 '01': {'avg_cov0': -7975574874343,
  'avg_cov1': -9233276233756,
  'avg_cov2': -7735453141090,
  'avg_cov3': -9511878980041},
 '10': {'avg_cov0': -10212445346678,
  'avg_cov1': -6901243634510,
  'avg_cov2': -10037161065654,
  'avg_cov3': -7237620927812},
 '11': {'avg_cov0': -8615534617261,
  'avg_cov1': -10731281563987,
  'avg_cov2': -8337682914534,
  'avg_cov3': -11027264497258}}

In [491]:
def get_rotation(qubit, angle, phase):
    return drag_hds[qubit].get_pulse_seq(angle, phase)

In [492]:
basis_states = ['00', '01', '10', '11'] 
opnames = ['avg_cov0', 'avg_cov1']
measurement_operators = {opname:np.diag([readout.states[basis_state][opname] for basis_state in basis_states]) for opname in opnames}
measurement_operators

{'avg_cov0': array([[ -8638209196591,               0,               0,
                       0],
        [              0,  -7975574874343,               0,
                       0],
        [              0,               0, -10212445346678,
                       0],
        [              0,               0,               0,
          -8615534617261]], dtype=int64),
 'avg_cov1': array([[ -6134102225210,               0,               0,
                       0],
        [              0,  -9233276233756,               0,
                       0],
        [              0,               0,  -6901243634510,
                       0],
        [              0,               0,               0,
         -10731281563987]], dtype=int64)}

In [493]:
imp.reload(multiqubit_tomography)

<module 'qsweepy.multiqubit_tomography' from 'C:\\qtlab_replacement\\qsweepy\\multiqubit_tomography.py'>

In [494]:
reconstruction_basis = {}
proj_seq = {}
import itertools

sigma_x = np.asarray([[0,1],  [1,0]])
sigma_y = np.asarray([[0,-1j],[1j,0]])
sigma_z = np.asarray([[1,0],  [0,-1]])
sigma_i = np.asarray([[1,0],  [0,1]])

cube_faces = {'+Z':(0.0, 0.0), 
              '+X':(np.pi/2., -np.pi/2.,),
              '+Y':(np.pi/2., 0.0),
              '-Z':(np.pi, 0.0),
              '-X':(np.pi/2., np.pi/2.),
              '-Y':(np.pi/2., np.pi)}

#cube_faces = {'+Z':(0.0, 0.0), '-Z':(np.pi, 0.0)}

cube_faces_unitaries = {k:    np.cos(v[0]/2)*sigma_i-\
                           1j*np.sin(v[0]/2)*np.cos(v[1])*sigma_x-\
                           1j*np.sin(v[0]/2)*np.sin(v[1])*sigma_y for k,v in cube_faces.items()}

#cube_faces_observable = {k:   np.dot(cube_faces_unitaries, )}

#for c1 in [(0., 0.), (np.pi/2., np.pi/2.), ()]
#for i,j in itertools.product([0,1], [0,1]):

initial_state = np.zeros((4,4))
initial_state[0,0] = 1.


#print (cube_faces_unitaries)
for q1_rotation_name, q1_rotation in cube_faces.items():
    for q2_rotation_name, q2_rotation in cube_faces.items():
        for q1_pre_name, q1_pre in cube_faces.items():
            for q2_pre_name, q2_pre in cube_faces.items():
                pulses_pre = get_rotation('1', *q1_pre)+get_rotation('2', *q2_pre)
                pulses = get_rotation('1', *q1_rotation)+get_rotation('2', *q2_rotation)+drag_hds['1'].tld.ro_sequence
                operators = {}
                #for ri,rj in itertools.product([0,1], [0,1]):
                for readout_result, _matrix in measurement_operators.items():
                    q1_unitary = cube_faces_unitaries[q1_rotation_name]
                    q2_unitary = cube_faces_unitaries[q2_rotation_name]
                    unitary = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary, q2_unitary), (4,4)), np.complex)
                    
                    q1_unitary_pre = cube_faces_unitaries[q1_pre_name]
                    q2_unitary_pre = cube_faces_unitaries[q2_pre_name]
                    unitary_pre = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary_pre, q2_unitary_pre), (4,4)), np.complex)
                    
                    initial = np.dot(unitary_pre, np.dot(initial_state, np.conj(unitary_pre).T)).ravel()
                    projection = np.dot(np.conj(unitary.T), np.dot(_matrix, unitary)).T.ravel()
                    
                    superoperator_projection = np.einsum('i,j->ij', projection, initial)
                    
                    operators[readout_result] = superoperator_projection
                proj_seq[q1_pre_name+q2_pre_name+q1_rotation_name+q2_rotation_name] = \
                    {'pre_pulses': pulses_pre, 'pulses':pulses, 'operators':operators}

single_qubit_chi_basis = np.arange(16)           
single_qubit_bases = ['I', 'X', 'Y', 'Z']
single_qubit_bases_operators = {'I':sigma_i, 'X':sigma_x, 'Y':sigma_y, 'Z':sigma_z}
for i,j in itertools.product(single_qubit_chi_basis, single_qubit_chi_basis):
    _matrix = np.zeros((16,16))
    _matrix[i,j]=1.
    reconstruction_basis[str(i)+','+str(j)] = {'operator': _matrix}


two_qubit_process_tomography = multiqubit_tomography.multiqubit_tomography(adc_reducer,
                                                                          pg, 
                                                                          proj_seq,
                                                                          reconstruction_basis)
two_qubit_process_tomography.readout_names = opnames#['00', '01', '01', '11']
two_qubit_process_tomography.confusion_matrix = np.identity(4)[:, 0:2]

In [495]:
two_qubit_process_tomography.set_prepare_seq([])
two_qubit_tomography_result = two_qubit_process_tomography.measure() 

+Z+Z+Z+Z
uncorreted: {'avg_cov0': -8709007644912, 'avg_cov1': -6027791035443, 'avg_cov2': -8553043896820, 'avg_cov3': -6324147594038}
corrected: {'avg_cov0': -8709007644912.0, 'avg_cov1': -6027791035443.0}
+Z+X+Z+Z
uncorreted: {'avg_cov0': -8220427402180, 'avg_cov1': -7811257259844, 'avg_cov2': -8018328167022, 'avg_cov3': -8096681514395}
corrected: {'avg_cov0': -8220427402180.0, 'avg_cov1': -7811257259844.0}
+Z+Y+Z+Z
uncorreted: {'avg_cov0': -8158199040407, 'avg_cov1': -7862531991639, 'avg_cov2': -7953601150383, 'avg_cov3': -8144998072109}
corrected: {'avg_cov0': -8158199040407.0, 'avg_cov1': -7862531991639.0}
+Z-Z+Z+Z
uncorreted: {'avg_cov0': -7842580994145, 'avg_cov1': -9203600402199, 'avg_cov2': -7604653727512, 'avg_cov3': -9478711650888}
corrected: {'avg_cov0': -7842580994145.0, 'avg_cov1': -9203600402199.0}
+Z-X+Z+Z
uncorreted: {'avg_cov0': -8294862917017, 'avg_cov1': -7718291413943, 'avg_cov2': -8095490824521, 'avg_cov3': -8004279925355}
corrected: {'avg_cov0': -8294862917017.0, 

+Z-X+Z+X
uncorreted: {'avg_cov0': -8589698656799, 'avg_cov1': -6215534737834, 'avg_cov2': -8429667728803, 'avg_cov3': -6508307269572}
corrected: {'avg_cov0': -8589698656799.0, 'avg_cov1': -6215534737834.0}
+Z-Y+Z+X
uncorreted: {'avg_cov0': -8070469337641, 'avg_cov1': -8338027367058, 'avg_cov2': -7853900130504, 'avg_cov3': -8619415332169}
corrected: {'avg_cov0': -8070469337641.0, 'avg_cov1': -8338027367058.0}
+X+Z+Z+X
uncorreted: {'avg_cov0': -8825619049782, 'avg_cov1': -8299663658742, 'avg_cov2': -8611259675169, 'avg_cov3': -8601084322774}
corrected: {'avg_cov0': -8825619049782.0, 'avg_cov1': -8299663658742.0}
+X+X+Z+X
uncorreted: {'avg_cov0': -8112622326255, 'avg_cov1': -10197551768932, 'avg_cov2': -7847659267964, 'avg_cov3': -10481533191812}
corrected: {'avg_cov0': -8112622326255.0, 'avg_cov1': -10197551768932.0}
+X+Y+Z+X
uncorreted: {'avg_cov0': -8867828296615, 'avg_cov1': -7980217592188, 'avg_cov2': -8662032469661, 'avg_cov3': -8283363615421}
corrected: {'avg_cov0': -8867828296615.

+X+Y+Z+Y
uncorreted: {'avg_cov0': -8119410165802, 'avg_cov1': -10272367104281, 'avg_cov2': -7851525019630, 'avg_cov3': -10555518791464}
corrected: {'avg_cov0': -8119410165802.0, 'avg_cov1': -10272367104281.0}
+X-Z+Z+Y
uncorreted: {'avg_cov0': -8788566812066, 'avg_cov1': -8482489993786, 'avg_cov2': -8568436921711, 'avg_cov3': -8783168158477}
corrected: {'avg_cov0': -8788566812066.0, 'avg_cov1': -8482489993786.0}
+X-X+Z+Y
uncorreted: {'avg_cov0': -9155060819216, 'avg_cov1': -7583844551417, 'avg_cov2': -8960332684200, 'avg_cov3': -7891886015766}
corrected: {'avg_cov0': -9155060819216.0, 'avg_cov1': -7583844551417.0}
+X-Y+Z+Y
uncorreted: {'avg_cov0': -9430346212753, 'avg_cov1': -6441077148160, 'avg_cov2': -9266025347316, 'avg_cov3': -6756977879498}
corrected: {'avg_cov0': -9430346212753.0, 'avg_cov1': -6441077148160.0}
+Y+Z+Z+Y
uncorreted: {'avg_cov0': -8818739776525, 'avg_cov1': -8426726012540, 'avg_cov2': -8602285657163, 'avg_cov3': -8727700620461}
corrected: {'avg_cov0': -8818739776525.

+Y+Z+Z-Z
uncorreted: {'avg_cov0': -8291260021000, 'avg_cov1': -10024242829163, 'avg_cov2': -8030056416503, 'avg_cov3': -10313525873375}
corrected: {'avg_cov0': -8291260021000.0, 'avg_cov1': -10024242829163.0}
+Y+X+Z-Z
uncorreted: {'avg_cov0': -8298082538819, 'avg_cov1': -9494582041246, 'avg_cov2': -8050151337385, 'avg_cov3': -9781108633064}
corrected: {'avg_cov0': -8298082538819.0, 'avg_cov1': -9494582041246.0}
+Y+Y+Z-Z
uncorreted: {'avg_cov0': -8647645856179, 'avg_cov1': -8615646820662, 'avg_cov2': -8423311345076, 'avg_cov3': -8909219629205}
corrected: {'avg_cov0': -8647645856179.0, 'avg_cov1': -8615646820662.0}
+Y-Z+Z-Z
uncorreted: {'avg_cov0': -9354377510245, 'avg_cov1': -6603744978152, 'avg_cov2': -9184447757310, 'avg_cov3': -6916807370870}
corrected: {'avg_cov0': -9354377510245.0, 'avg_cov1': -6603744978152.0}
+Y-X+Z-Z
uncorreted: {'avg_cov0': -8932933031645, 'avg_cov1': -7551045505471, 'avg_cov2': -8737270120681, 'avg_cov3': -7854893936381}
corrected: {'avg_cov0': -8932933031645.

+Y-X+Z-X
uncorreted: {'avg_cov0': -8115902679709, 'avg_cov1': -10249336348873, 'avg_cov2': -7850399680047, 'avg_cov3': -10532221400645}
corrected: {'avg_cov0': -8115902679709.0, 'avg_cov1': -10249336348873.0}
+Y-Y+Z-X
uncorreted: {'avg_cov0': -8817280849006, 'avg_cov1': -8018480826817, 'avg_cov2': -8609285720701, 'avg_cov3': -8317555945444}
corrected: {'avg_cov0': -8817280849006.0, 'avg_cov1': -8018480826817.0}
-Z+Z+Z-X
uncorreted: {'avg_cov0': -9337439062002, 'avg_cov1': -8846571478227, 'avg_cov2': -9108728140089, 'avg_cov3': -9161548125915}
corrected: {'avg_cov0': -9337439062002.0, 'avg_cov1': -8846571478227.0}
-Z+X+Z-X
uncorreted: {'avg_cov0': -9805812637503, 'avg_cov1': -7319707979804, 'avg_cov2': -9618284262051, 'avg_cov3': -7646306717697}
corrected: {'avg_cov0': -9805812637503.0, 'avg_cov1': -7319707979804.0}
-Z+Y+Z-X
uncorreted: {'avg_cov0': -9021520122386, 'avg_cov1': -9610694997151, 'avg_cov2': -8773749928510, 'avg_cov3': -9918198212327}
corrected: {'avg_cov0': -9021520122386.

...

+Z-X-Y+X
uncorreted: {'avg_cov0': -9401538333497, 'avg_cov1': -6637367810677, 'avg_cov2': -9229830579851, 'avg_cov3': -6952829293861}
corrected: {'avg_cov0': -9401538333497.0, 'avg_cov1': -6637367810677.0}
+Z-Y-Y+X
uncorreted: {'avg_cov0': -8531359513768, 'avg_cov1': -9071728013801, 'avg_cov2': -8298411463253, 'avg_cov3': -9365660012536}
corrected: {'avg_cov0': -8531359513768.0, 'avg_cov1': -9071728013801.0}
+X+Z-Y+X
uncorreted: {'avg_cov0': -8778382550551, 'avg_cov1': -8100037068051, 'avg_cov2': -8571329605493, 'avg_cov3': -8399508736556}
corrected: {'avg_cov0': -8778382550551.0, 'avg_cov1': -8100037068051.0}
+X+X-Y+X
uncorreted: {'avg_cov0': -7967509877255, 'avg_cov1': -10075646066590, 'avg_cov2': -7707487544606, 'avg_cov3': -10354768989169}
corrected: {'avg_cov0': -7967509877255.0, 'avg_cov1': -10075646066590.0}
+X+Y-Y+X
uncorreted: {'avg_cov0': -8592624539709, 'avg_cov1': -7924910483292, 'avg_cov2': -8387589443138, 'avg_cov3': -8219923632303}
corrected: {'avg_cov0': -8592624539709.

+X+Y-Y+Y
uncorreted: {'avg_cov0': -8105113625134, 'avg_cov1': -10027590500230, 'avg_cov2': -7843333672941, 'avg_cov3': -10309773336514}
corrected: {'avg_cov0': -8105113625134.0, 'avg_cov1': -10027590500230.0}
+X-Z-Y+Y
uncorreted: {'avg_cov0': -8611744780130, 'avg_cov1': -7995229488053, 'avg_cov2': -8403394916466, 'avg_cov3': -8288838124414}
corrected: {'avg_cov0': -8611744780130.0, 'avg_cov1': -7995229488053.0}
+X-X-Y+Y
uncorreted: {'avg_cov0': -8663453693022, 'avg_cov1': -7829314368547, 'avg_cov2': -8459006199492, 'avg_cov3': -8126715085986}
corrected: {'avg_cov0': -8663453693022.0, 'avg_cov1': -7829314368547.0}
+X-Y-Y+Y
uncorreted: {'avg_cov0': -9242269716191, 'avg_cov1': -6387847879706, 'avg_cov2': -9076429649267, 'avg_cov3': -6699421798780}
corrected: {'avg_cov0': -9242269716191.0, 'avg_cov1': -6387847879706.0}
+Y+Z-Y+Y
uncorreted: {'avg_cov0': -8437321953383, 'avg_cov1': -7720341265252, 'avg_cov2': -8235934995390, 'avg_cov3': -8010840561057}
corrected: {'avg_cov0': -8437321953383.

+Y+Z-Y-Z
uncorreted: {'avg_cov0': -7959068638524, 'avg_cov1': -9382020008197, 'avg_cov2': -7717094141700, 'avg_cov3': -9659984890629}
corrected: {'avg_cov0': -7959068638524.0, 'avg_cov1': -9382020008197.0}
+Y+X-Y-Z
uncorreted: {'avg_cov0': -8015178135847, 'avg_cov1': -9090399462153, 'avg_cov2': -7778953311728, 'avg_cov3': -9371181043164}
corrected: {'avg_cov0': -8015178135847.0, 'avg_cov1': -9090399462153.0}
+Y+Y-Y-Z
uncorreted: {'avg_cov0': -8285708089923, 'avg_cov1': -7765403312576, 'avg_cov2': -8084525067574, 'avg_cov3': -8051783438023}
corrected: {'avg_cov0': -8285708089923.0, 'avg_cov1': -7765403312576.0}
+Y-Z-Y-Z
uncorreted: {'avg_cov0': -8898115236411, 'avg_cov1': -6125023488132, 'avg_cov2': -8741448839274, 'avg_cov3': -6427560115519}
corrected: {'avg_cov0': -8898115236411.0, 'avg_cov1': -6125023488132.0}
+Y-X-Y-Z
uncorreted: {'avg_cov0': -8622782718143, 'avg_cov1': -6837721800913, 'avg_cov2': -8444088447880, 'avg_cov3': -7131686430347}
corrected: {'avg_cov0': -8622782718143.0, 

+Y-X-Y-X
uncorreted: {'avg_cov0': -7944375194351, 'avg_cov1': -9520363790576, 'avg_cov2': -7697439157121, 'avg_cov3': -9797859070627}
corrected: {'avg_cov0': -7944375194351.0, 'avg_cov1': -9520363790576.0}
+Y-Y-Y-X
uncorreted: {'avg_cov0': -8410896572302, 'avg_cov1': -7372329444581, 'avg_cov2': -8219227644002, 'avg_cov3': -7660248040035}
corrected: {'avg_cov0': -8410896572302.0, 'avg_cov1': -7372329444581.0}
-Z+Z-Y-X
uncorreted: {'avg_cov0': -8760214287879, 'avg_cov1': -8377042036432, 'avg_cov2': -8543213806351, 'avg_cov3': -8676249057195}
corrected: {'avg_cov0': -8760214287879.0, 'avg_cov1': -8377042036432.0}
-Z+X-Y-X
uncorreted: {'avg_cov0': -9525054528463, 'avg_cov1': -6525918500981, 'avg_cov2': -9358616506026, 'avg_cov3': -6843810436034}
corrected: {'avg_cov0': -9525054528463.0, 'avg_cov1': -6525918500981.0}
-Z+Y-Y-X
uncorreted: {'avg_cov0': -8755281813378, 'avg_cov1': -8849131904234, 'avg_cov2': -8523951217076, 'avg_cov3': -9148291024231}
corrected: {'avg_cov0': -8755281813378.0, 

-Z+Y-Y-Y
uncorreted: {'avg_cov0': -9448553630991, 'avg_cov1': -6776943226745, 'avg_cov2': -9276101658758, 'avg_cov3': -7094298355833}
corrected: {'avg_cov0': -9448553630991.0, 'avg_cov1': -6776943226745.0}
-Z-Z-Y-Y
uncorreted: {'avg_cov0': -8639834986222, 'avg_cov1': -8910329457292, 'avg_cov2': -8412149333044, 'avg_cov3': -9205723955169}
corrected: {'avg_cov0': -8639834986222.0, 'avg_cov1': -8910329457292.0}
-Z-X-Y-Y
uncorreted: {'avg_cov0': -8520986813367, 'avg_cov1': -9089735517743, 'avg_cov2': -8288181939910, 'avg_cov3': -9381287464292}
corrected: {'avg_cov0': -8520986813367.0, 'avg_cov1': -9089735517743.0}
-Z-Y-Y-Y
uncorreted: {'avg_cov0': -8174959733169, 'avg_cov1': -10372958978195, 'avg_cov2': -7908720813359, 'avg_cov3': -10657191015722}
corrected: {'avg_cov0': -8174959733169.0, 'avg_cov1': -10372958978195.0}
-X+Z-Y-Y
uncorreted: {'avg_cov0': -8791027995607, 'avg_cov1': -8473675619884, 'avg_cov2': -8573809138312, 'avg_cov3': -8773635512049}
corrected: {'avg_cov0': -8791027995607.

In [496]:
pqjson.dump(type="identity-tomography",name='20190508-2',params = two_qubit_tomography_result)

In [497]:
pqjson.dump(type='measurement-operators', name='20190508-2', params=measurement_operators)

In [498]:
cube_state_prod = [i1+i2 for i1, i2 in itertools.product(['+X', '-X', '+Y', '-Y', '+Z', '-Z'], 
                                    ['+X', '-X', '+Y', '-Y', '+Z', '-Z'])]
m0 = np.zeros((36,36))
m1 = np.zeros((36,36))
for left_state_id, left_state in enumerate(cube_state_prod):
    for right_state_id, right_state in enumerate(cube_state_prod):
        m0[left_state_id, right_state_id] = two_qubit_tomography_result[left_state+right_state+'-Pavg_cov0']
        m1[left_state_id, right_state_id] = two_qubit_tomography_result[left_state+right_state+'-Pavg_cov1']

figs, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].pcolormesh(m0)
axes[0].set_title('avg_cov0')
axes[0].grid(True)
axes[0].set_xticks(np.arange(36))
axes[0].set_yticks(np.arange(36)[::-1])
axes[1].pcolormesh(m1)
axes[1].set_title('avg_cov1')
axes[1].grid(True)
axes[1].set_xticks(np.arange(36))
axes[1].set_yticks(np.arange(36)[::-1])
fig = plt.gcf()
fig.canvas.set_window_title('Raw measurements identity measured')

In [559]:
m_identity = np.zeros((16,16), dtype=np.complex)
for left_state_id, left_state in enumerate(np.arange(16)):
    for right_state_id, right_state in enumerate(np.arange(16)):
        m_identity[left_state_id, right_state_id] = two_qubit_tomography_result[str(left_state)+','+str(right_state)]

In [560]:
fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix identity measured')
axes[0].pcolormesh(np.rot90(np.real(m_identity)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])
axes[0].grid(True)

axes[1].pcolormesh(np.rot90(np.imag(m_identity)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
axes[1].grid(True)
#plt.colorbar()

In [514]:
#rotation_matrix=np.reshape(np.einsum('ij,kl->ikjl', [[0,1],[1,0]],np.identity(2)), (4,4))
rotation_matrix = np.identity(4)
rotation_matrix_process = np.reshape(np.einsum('ij,kl->ikjl', np.conj(rotation_matrix),rotation_matrix), (16,16))

fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix SDP identity expected')
axes[0].pcolormesh(np.rot90(np.real(rotation_matrix_process)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])

axes[1].pcolormesh(np.rot90(np.imag(rotation_matrix_process)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
#plt.colorbar()
plt.grid()

In [534]:
np.trace(np.dot(rotation_matrix_process, m_identity))/16

(0.859300193846513+1.5959455978986625e-16j)

In [311]:
np.trace(np.dot(rotation_matrix_process, m_identity))/16

(0.5733229581053078-3.191891195797325e-16j)

In [530]:
def process_tomography_measurement_results_to_chi_matrix(measurement_results):
    import cvxopt
    import cvxpy
    chi_dimension = 16
    initial_state = np.zeros((4,4))
    initial_state[0,0] = 1.
    initials = np.zeros((16, 6*6), dtype=np.complex)
    measurement_operators_ = np.zeros((6*6*2, 16), dtype=np.complex)
    measurement_results_np = np.zeros((6*6*2,6*6))
    projection_order = ['+Z', '+X', '+Y', '-Z', '-X', '-Y']
    for p3_id, p3 in enumerate(projection_order):
        for p4_id, p4 in enumerate(projection_order):
            for p1_id, p1 in enumerate(projection_order):
                for p2_id, p2 in enumerate(projection_order):
                    initial_id = p1_id*6+p2_id
                    projection_id = p3_id*2*6+p4_id*2
                    for readout_result, _matrix in measurement_operators.items():
                        _matrix2 = _matrix*1e-13+0.8
                        meas = measurement_results[p1+p2+p3+p4+'-P'+readout_result]*1e-13+0.8
                        measurement_results_np[projection_id, initial_id] = meas
                        projection_id+=1
                    
    for p3_id, p3 in enumerate(projection_order):
        for p4_id, p4 in enumerate(projection_order):
            projection_id = p3_id*2*6+p4_id*2
            for readout_result, _matrix in measurement_operators.items():
                _matrix2 = _matrix*1e-13+0.8
                q1_unitary = cube_faces_unitaries[p3]
                q2_unitary = cube_faces_unitaries[p4]
                unitary = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary, q2_unitary), (4,4)), np.complex)
                projection = ((np.dot(np.conj(unitary.T), np.dot(_matrix2, unitary))).T).ravel()
                measurement_operators_[projection_id, :] = projection
                projection_id +=1

    for p1_id, p1 in enumerate(projection_order):
        for p2_id, p2 in enumerate(projection_order):
            initial_id = p1_id*6+p2_id
            q1_unitary_pre = cube_faces_unitaries[p1]
            q2_unitary_pre = cube_faces_unitaries[p2]
            unitary_pre = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary_pre, q2_unitary_pre), (4,4)), np.complex)
            initial = np.dot(unitary_pre, np.dot(initial_state, np.conj(unitary_pre).T)).ravel()
                
            initials[:, initial_id] = initial
            
    x = cvxpy.Variable((chi_dimension,chi_dimension), complex=True)
    lstsq_objective = cvxpy.atoms.sum_squares(cvxpy.abs(measurement_operators_@x@initials-measurement_results_np))
    
    #i_indeces = [int(_id/(4*4*4))+         ((_id)%4)*4 for _id in range(4*4*4*4)]
    #j_indeces = [(int(_id/(4))%4)+(int(_id/(4*4))%4)*4 for _id in range(4*4*4*4)]
    i_indeces = [int(_id%(4))+         (int(_id/4/4)%4)*4 for _id in range(4*4*4*4)]
    j_indeces = [(int(_id/(4))%4)+(int(_id/(4*4*4))%4)*4 for _id in range(4*4*4*4)]
    #i_indeces = [int(_id/(4*4*4))+         (int(_id/4)%4)*4 for _id in range(4*4*4*4)]
    #j_indeces = [(int(_id/(4*4))%4)+(int(_id%4)*4) for _id in range(4*4*4*4)]
    x_reshaped = cvxpy.reshape(x[i_indeces, j_indeces], (16,16))
    x_reshaped = cvxpy.reshape(x, (16,16))
    
    psd_constraint = x_reshaped>>0
    hermitian_constraint = x_reshaped.H==x_reshaped
    
    # Create two constraints.
    constraints = [psd_constraint, hermitian_constraint]
    
    # Form objective.
    obj = cvxpy.Minimize(lstsq_objective)
    # Form and solve problem.
    prob = cvxpy.Problem(obj, constraints)
    prob.solve(solver=cvxpy.CVXOPT, verbose=True)
    #print (x_reshaped.value)
    return x.value

In [531]:
m_identity_sdp = process_tomography_measurement_results_to_chi_matrix(two_qubit_tomography_result)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.0000e+00  3e+03  1e+01  5e+01  1e+00
 1:  3.0103e+00  1.9973e+00  9e+02  6e+00  3e+01  5e-03
 2:  4.6431e+00  4.2429e+00  1e+02  2e+00  1e+01  2e-03
 3:  5.8215e+00  5.5793e+00  7e+01  1e+00  6e+00  2e-03
 4:  9.0056e+00  8.9180e+00  3e+01  5e-01  2e+00  5e-04
 5:  1.1190e+01  1.1149e+01  1e+01  2e-01  1e+00  5e-04
 6:  1.1434e+01  1.1394e+01  1e+01  2e-01  1e+00  8e-04
 7:  1.2764e+01  1.2754e+01  3e+00  6e-02  3e-01  3e-04
 8:  1.2929e+01  1.2922e+01  2e+00  4e-02  2e-01  3e-04
 9:  1.3246e+01  1.3245e+01  1e-01  2e-03  1e-02  2e-05
10:  1.3263e+01  1.3263e+01  9e-03  2e-04  9e-04  2e-06
11:  1.3265e+01  1.3265e+01  2e-03  4e-05  2e-04  4e-07
12:  1.3265e+01  1.3265e+01  7e-04  1e-05  6e-05  1e-07
13:  1.3265e+01  1.3265e+01  9e-05  2e-06  9e-06  2e-08
14:  1.3265e+01  1.3265e+01  1e-05  3e-07  1e-06  3e-09
15:  1.3265e+01  1.3265e+01  1e-06  2e-08  1e-07  3e-10
Optimal solution found.


In [532]:
fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix SDP identity measured')
axes[0].pcolormesh(np.rot90(np.real(m_identity_sdp)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])

axes[1].pcolormesh(np.rot90(np.imag(m_identity_sdp)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
#plt.colorbar()
plt.grid()

In [533]:
np.trace(np.dot(rotation_matrix_process, m_identity_sdp))/16

(0.4274084745455393+0j)

In [501]:
pg.channels['iq_ex_00-2-11'].set_if(4.1338e9-lo1.get_frequency())

In [502]:
bswap_pulse = pg.pmulti(780e-9, ('iq_ex_00-2-11', pg.rect, 0.2))

In [503]:
two_qubit_process_tomography.set_prepare_seq([bswap_pulse])
bswap_tomography = two_qubit_process_tomography.measure() 

+Z+Z+Z+Z
uncorreted: {'avg_cov0': -8392273600678, 'avg_cov1': -9835092454802, 'avg_cov2': -8137927198403, 'avg_cov3': -10125419714590}
corrected: {'avg_cov0': -8392273600678.0, 'avg_cov1': -9835092454802.0}
+Z+X+Z+Z
uncorreted: {'avg_cov0': -7989402791197, 'avg_cov1': -9897030683707, 'avg_cov2': -7731942604156, 'avg_cov3': -10176082396702}
corrected: {'avg_cov0': -7989402791197.0, 'avg_cov1': -9897030683707.0}
+Z+Y+Z+Z
uncorreted: {'avg_cov0': -8157819328900, 'avg_cov1': -9750888373379, 'avg_cov2': -7906258760050, 'avg_cov3': -10035820160092}
corrected: {'avg_cov0': -8157819328900.0, 'avg_cov1': -9750888373379.0}
+Z-Z+Z+Z
uncorreted: {'avg_cov0': -7855665662189, 'avg_cov1': -9584073143388, 'avg_cov2': -7607505161017, 'avg_cov3': -9860639346040}
corrected: {'avg_cov0': -7855665662189.0, 'avg_cov1': -9584073143388.0}
+Z-X+Z+Z
uncorreted: {'avg_cov0': -8237565492463, 'avg_cov1': -9703892745970, 'avg_cov2': -7985620743393, 'avg_cov3': -9990721879314}
corrected: {'avg_cov0': -8237565492463.

+Z-X+Z+X
uncorreted: {'avg_cov0': -8703611765299, 'avg_cov1': -9100906326248, 'avg_cov2': -8469809142943, 'avg_cov3': -9400326910641}
corrected: {'avg_cov0': -8703611765299.0, 'avg_cov1': -9100906326248.0}
+Z-Y+Z+X
uncorreted: {'avg_cov0': -8379526856392, 'avg_cov1': -8674121099701, 'avg_cov2': -8156158887502, 'avg_cov3': -8964280795918}
corrected: {'avg_cov0': -8379526856392.0, 'avg_cov1': -8674121099701.0}
+X+Z+Z+X
uncorreted: {'avg_cov0': -9339851323055, 'avg_cov1': -8214564076735, 'avg_cov2': -9129681336915, 'avg_cov3': -8530660048487}
corrected: {'avg_cov0': -9339851323055.0, 'avg_cov1': -8214564076735.0}
+X+X+Z+X
uncorreted: {'avg_cov0': -8786667798512, 'avg_cov1': -8493837189781, 'avg_cov2': -8566413409900, 'avg_cov3': -8793372394178}
corrected: {'avg_cov0': -8786667798512.0, 'avg_cov1': -8493837189781.0}
+X+Y+Z+X
uncorreted: {'avg_cov0': -8977962610275, 'avg_cov1': -8486526164041, 'avg_cov2': -8759519003434, 'avg_cov3': -8792209910845}
corrected: {'avg_cov0': -8977962610275.0, 

+X+Y+Z+Y
uncorreted: {'avg_cov0': -8489500054576, 'avg_cov1': -9503492963186, 'avg_cov2': -8245259810115, 'avg_cov3': -9797211750199}
corrected: {'avg_cov0': -8489500054576.0, 'avg_cov1': -9503492963186.0}
+X-Z+Z+Y
uncorreted: {'avg_cov0': -8179213722430, 'avg_cov1': -9640778082506, 'avg_cov2': -7930838336883, 'avg_cov3': -9925649362872}
corrected: {'avg_cov0': -8179213722430.0, 'avg_cov1': -9640778082506.0}
+X-X+Z+Y
uncorreted: {'avg_cov0': -8752863647546, 'avg_cov1': -8920462317651, 'avg_cov2': -8524562109967, 'avg_cov3': -9218594925030}
corrected: {'avg_cov0': -8752863647546.0, 'avg_cov1': -8920462317651.0}
+X-Y+Z+Y
uncorreted: {'avg_cov0': -8650470248843, 'avg_cov1': -9106359464544, 'avg_cov2': -8416324502370, 'avg_cov3': -9403679927717}
corrected: {'avg_cov0': -8650470248843.0, 'avg_cov1': -9106359464544.0}
+Y+Z+Z+Y
uncorreted: {'avg_cov0': -8719170276667, 'avg_cov1': -9898742014509, 'avg_cov2': -8464172401137, 'avg_cov3': -10197329646896}
corrected: {'avg_cov0': -8719170276667.0,

+Y+Z+Z-Z
uncorreted: {'avg_cov0': -8288604760905, 'avg_cov1': -10454395448280, 'avg_cov2': -8018461245770, 'avg_cov3': -10743977844833}
corrected: {'avg_cov0': -8288604760905.0, 'avg_cov1': -10454395448280.0}
+Y+X+Z-Z
uncorreted: {'avg_cov0': -8170666109128, 'avg_cov1': -10032666210922, 'avg_cov2': -7913247157054, 'avg_cov3': -10318940024535}
corrected: {'avg_cov0': -8170666109128.0, 'avg_cov1': -10032666210922.0}
+Y+Y+Z-Z
uncorreted: {'avg_cov0': -8096834111572, 'avg_cov1': -10100637288241, 'avg_cov2': -7836273248931, 'avg_cov3': -10383545163616}
corrected: {'avg_cov0': -8096834111572.0, 'avg_cov1': -10100637288241.0}
+Y-Z+Z-Z
uncorreted: {'avg_cov0': -8074244122306, 'avg_cov1': -9618949935847, 'avg_cov2': -7825948463200, 'avg_cov3': -9900466224050}
corrected: {'avg_cov0': -8074244122306.0, 'avg_cov1': -9618949935847.0}
+Y-X+Z-Z
uncorreted: {'avg_cov0': -8034524680763, 'avg_cov1': -10448239526440, 'avg_cov2': -7765508779668, 'avg_cov3': -10729118619229}
corrected: {'avg_cov0': -803452

+Y-X+Z-X
uncorreted: {'avg_cov0': -8626687842146, 'avg_cov1': -8816757463776, 'avg_cov2': -8396939737900, 'avg_cov3': -9112323849659}
corrected: {'avg_cov0': -8626687842146.0, 'avg_cov1': -8816757463776.0}
+Y-Y+Z-X
uncorreted: {'avg_cov0': -8492518883150, 'avg_cov1': -9133483240557, 'avg_cov2': -8255847264022, 'avg_cov3': -9426343651645}
corrected: {'avg_cov0': -8492518883150.0, 'avg_cov1': -9133483240557.0}
-Z+Z+Z-X
uncorreted: {'avg_cov0': -8957062381197, 'avg_cov1': -9202309285641, 'avg_cov2': -8719912345067, 'avg_cov3': -9507706863000}
corrected: {'avg_cov0': -8957062381197.0, 'avg_cov1': -9202309285641.0}
-Z+X+Z-X
uncorreted: {'avg_cov0': -8611700061898, 'avg_cov1': -9301008660755, 'avg_cov2': -8369388891233, 'avg_cov3': -9596797490484}
corrected: {'avg_cov0': -8611700061898.0, 'avg_cov1': -9301008660755.0}
-Z+Y+Z-X
uncorreted: {'avg_cov0': -8590283887839, 'avg_cov1': -9465793709472, 'avg_cov2': -8343800732068, 'avg_cov3': -9761480002820}
corrected: {'avg_cov0': -8590283887839.0, 

...

+Z-X-Y+X
uncorreted: {'avg_cov0': -8458585011999, 'avg_cov1': -8897557183100, 'avg_cov2': -8228368458816, 'avg_cov3': -9188637346972}
corrected: {'avg_cov0': -8458585011999.0, 'avg_cov1': -8897557183100.0}
+Z-Y-Y+X
uncorreted: {'avg_cov0': -8824342200731, 'avg_cov1': -8917840830192, 'avg_cov2': -8594947674116, 'avg_cov3': -9219926924593}
corrected: {'avg_cov0': -8824342200731.0, 'avg_cov1': -8917840830192.0}
+X+Z-Y+X
uncorreted: {'avg_cov0': -8924622124203, 'avg_cov1': -8512680838357, 'avg_cov2': -8705100706486, 'avg_cov3': -8815424751795}
corrected: {'avg_cov0': -8924622124203.0, 'avg_cov1': -8512680838357.0}
+X+X-Y+X
uncorreted: {'avg_cov0': -8947429318976, 'avg_cov1': -8561604848388, 'avg_cov2': -8726099572989, 'avg_cov3': -8865500337547}
corrected: {'avg_cov0': -8947429318976.0, 'avg_cov1': -8561604848388.0}
+X+Y-Y+X
uncorreted: {'avg_cov0': -8549657098090, 'avg_cov1': -8699338985749, 'avg_cov2': -8326863130447, 'avg_cov3': -8992256271213}
corrected: {'avg_cov0': -8549657098090.0, 

+X+Y-Y+Y
uncorreted: {'avg_cov0': -8378806842203, 'avg_cov1': -9265453977052, 'avg_cov2': -8141734891200, 'avg_cov3': -9553851899172}
corrected: {'avg_cov0': -8378806842203.0, 'avg_cov1': -9265453977052.0}
+X-Z-Y+Y
uncorreted: {'avg_cov0': -8352537702729, 'avg_cov1': -9795914157174, 'avg_cov2': -8098529759016, 'avg_cov3': -10083749984347}
corrected: {'avg_cov0': -8352537702729.0, 'avg_cov1': -9795914157174.0}
+X-X-Y+Y
uncorreted: {'avg_cov0': -8640941378080, 'avg_cov1': -9038980129460, 'avg_cov2': -8407234576496, 'avg_cov3': -9334093962026}
corrected: {'avg_cov0': -8640941378080.0, 'avg_cov1': -9038980129460.0}
+X-Y-Y+Y
uncorreted: {'avg_cov0': -8902401248342, 'avg_cov1': -9394704877988, 'avg_cov2': -8659360345198, 'avg_cov3': -9697362275683}
corrected: {'avg_cov0': -8902401248342.0, 'avg_cov1': -9394704877988.0}
+Y+Z-Y+Y
uncorreted: {'avg_cov0': -8535311171514, 'avg_cov1': -9567202461414, 'avg_cov2': -8290281722407, 'avg_cov3': -9861431258499}
corrected: {'avg_cov0': -8535311171514.0,

+Y+Z-Y-Z
uncorreted: {'avg_cov0': -8272873675057, 'avg_cov1': -10029254041120, 'avg_cov2': -8013401085061, 'avg_cov3': -10315662327676}
corrected: {'avg_cov0': -8272873675057.0, 'avg_cov1': -10029254041120.0}
+Y+X-Y-Z
uncorreted: {'avg_cov0': -8380200945204, 'avg_cov1': -9951003789495, 'avg_cov2': -8121600953630, 'avg_cov3': -10241665305270}
corrected: {'avg_cov0': -8380200945204.0, 'avg_cov1': -9951003789495.0}
+Y+Y-Y-Z
uncorreted: {'avg_cov0': -8211773847910, 'avg_cov1': -9748892485727, 'avg_cov2': -7959723599950, 'avg_cov3': -10034480952174}
corrected: {'avg_cov0': -8211773847910.0, 'avg_cov1': -9748892485727.0}
+Y-Z-Y-Z
uncorreted: {'avg_cov0': -8284725264955, 'avg_cov1': -9796883040046, 'avg_cov2': -8029081709539, 'avg_cov3': -10084513325766}
corrected: {'avg_cov0': -8284725264955.0, 'avg_cov1': -9796883040046.0}
+Y-X-Y-Z
uncorreted: {'avg_cov0': -8159910398382, 'avg_cov1': -10241488469659, 'avg_cov2': -7895083848473, 'avg_cov3': -10527489648764}
corrected: {'avg_cov0': -815991039

+Y-X-Y-X
uncorreted: {'avg_cov0': -8503609704828, 'avg_cov1': -9292887621827, 'avg_cov2': -8262639986441, 'avg_cov3': -9586098289735}
corrected: {'avg_cov0': -8503609704828.0, 'avg_cov1': -9292887621827.0}
+Y-Y-Y-X
uncorreted: {'avg_cov0': -8804697965138, 'avg_cov1': -9299328570812, 'avg_cov2': -8565346420145, 'avg_cov3': -9599846521033}
corrected: {'avg_cov0': -8804697965138.0, 'avg_cov1': -9299328570812.0}
-Z+Z-Y-X
uncorreted: {'avg_cov0': -8698610338092, 'avg_cov1': -8876131976461, 'avg_cov2': -8470778599658, 'avg_cov3': -9175369170416}
corrected: {'avg_cov0': -8698610338092.0, 'avg_cov1': -8876131976461.0}
-Z+X-Y-X
uncorreted: {'avg_cov0': -8489894552875, 'avg_cov1': -9195216055893, 'avg_cov2': -8251561008413, 'avg_cov3': -9489397429371}
corrected: {'avg_cov0': -8489894552875.0, 'avg_cov1': -9195216055893.0}
-Z+Y-Y-X
uncorreted: {'avg_cov0': -8190078109269, 'avg_cov1': -9194304551025, 'avg_cov2': -7950612556472, 'avg_cov3': -9480204339669}
corrected: {'avg_cov0': -8190078109269.0, 

-Z+Y-Y-Y
uncorreted: {'avg_cov0': -8136287663047, 'avg_cov1': -9763622060329, 'avg_cov2': -7879931211620, 'avg_cov3': -10047645309022}
corrected: {'avg_cov0': -8136287663047.0, 'avg_cov1': -9763622060329.0}
-Z-Z-Y-Y
uncorreted: {'avg_cov0': -8145418378024, 'avg_cov1': -9688641245157, 'avg_cov2': -7892700022746, 'avg_cov3': -9973256364754}
corrected: {'avg_cov0': -8145418378024.0, 'avg_cov1': -9688641245157.0}
-Z-X-Y-Y
uncorreted: {'avg_cov0': -8248650380273, 'avg_cov1': -9862469543426, 'avg_cov2': -7992093824612, 'avg_cov3': -10150356281379}
corrected: {'avg_cov0': -8248650380273.0, 'avg_cov1': -9862469543426.0}
-Z-Y-Y-Y
uncorreted: {'avg_cov0': -8617828091929, 'avg_cov1': -9773043687222, 'avg_cov2': -8363695674722, 'avg_cov3': -10068949337652}
corrected: {'avg_cov0': -8617828091929.0, 'avg_cov1': -9773043687222.0}
-X+Z-Y-Y
uncorreted: {'avg_cov0': -8605184287971, 'avg_cov1': -8957621212770, 'avg_cov2': -8370747804934, 'avg_cov3': -9253806772850}
corrected: {'avg_cov0': -8605184287971.

In [535]:
m0_bswap = np.zeros((36,36))
m1_bswap = np.zeros((36,36))
for left_state_id, left_state in enumerate(cube_state_prod):
    for right_state_id, right_state in enumerate(cube_state_prod):
        m0_bswap[left_state_id, right_state_id] = bswap_tomography[left_state+right_state+'-Pavg_cov0']
        m1_bswap[left_state_id, right_state_id] = bswap_tomography[left_state+right_state+'-Pavg_cov1']

figs, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].pcolormesh(m0_bswap)
axes[0].set_title('avg_cov0')
axes[0].grid(True)
axes[0].set_xticks(np.arange(36))
axes[0].set_yticks(np.arange(36)[::-1])
axes[1].pcolormesh(m1_bswap)
axes[1].set_title('avg_cov1')
axes[1].grid(True)
axes[1].set_xticks(np.arange(36))
axes[1].set_yticks(np.arange(36)[::-1])
fig = plt.gcf()
fig.canvas.set_window_title('Raw measurements bSWAP measured')

In [556]:
m_bswap = np.zeros((16,16), dtype=np.complex)
for left_state_id, left_state in enumerate(np.arange(16)):
    for right_state_id, right_state in enumerate(np.arange(16)):
        m_bswap[left_state_id, right_state_id] = bswap_tomography[str(left_state)+','+str(right_state)]

In [557]:
fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix measured')
axes[0].pcolormesh(np.rot90(np.real(m_bswap)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])
axes[0].grid(True)

axes[1].pcolormesh(np.rot90(np.imag(m_bswap)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
axes[1].grid(True)

In [507]:
rotation_matrix = [[0,0,0,-1j],[0,1,0,0],[0,0,1,0],[-1j,0,0,0]]
rotation_matrix_process = np.reshape(np.einsum('ij,kl->ikjl', np.conj(rotation_matrix),rotation_matrix), (16,16))

fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix SDP identity expected')
axes[0].pcolormesh(np.rot90(np.real(rotation_matrix_process)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])

axes[1].pcolormesh(np.rot90(np.imag(rotation_matrix_process)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
#plt.colorbar()
plt.grid()

In [508]:
cy=np.asarray([[1,0,0,0],[0,1,0,0],[0,0,0,-1j],[0,0,1j,0]])
cx=np.asarray([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
def process_matrix(process, process_type='chi', plot=False, plot_name=''):
    import numpy as np
    import itertools
    operators_1Q = { 'I': np.matrix([[1,0],[0,1]]),
                 'X': np.matrix([[0,1],[1,0]]),
                  'Y': np.matrix([[0,-1j],[1j,0]]),
                  'Z': np.matrix([[1,0],[0,-1]])}
    operators_2Q=dict()
    for i,j in operators_1Q.items():
        for k,l in operators_1Q.items():
            operators_2Q[i+k]=np.kron(j,l)
    if process_type=='chi':
        process_tensor = np.reshape(process, (4,4,4,4))
    matrix={}
    for ind1,op1 in operators_2Q.items():
        for ind2,op2 in operators_2Q.items():
            if process_type=='chi':
                matrix[ind1,ind2]=0.25*np.einsum('ijkl,ij,kl->', process_tensor, op1, op2)
            else:
                matrix[ind1,ind2]=(0.25*np.trace(op1@process@op2@np.conj(process.T)))
                
    basis = ['I','X','Y','Z']
    double_basis = [i+j for i,j in itertools.product(basis, basis)]
    double_basis
    matrix_np = np.zeros((16,16))
    for i,f in itertools.product(double_basis, double_basis):
        i_id = double_basis.index(i)
        f_id = double_basis.index(f)
        matrix_np[i_id, f_id] = matrix[(i,f)]
    if plot:
        plt.figure()
        plt.pcolormesh(np.rot90(matrix_np),cmap='RdBu',vmax=1,vmin=-1)
        plt.xticks(np.arange(len(double_basis))+0.5,double_basis)
        plt.yticks(np.arange(len(double_basis))+0.5,double_basis[::-1])
        plt.colorbar()
        plt.grid()
        fig = plt.gcf()
        fig.canvas.set_window_title('Pauli transfer matrix '+plot_name)
    return matrix_np

In [538]:
m_bswap_sdp = process_tomography_measurement_results_to_chi_matrix(bswap_tomography)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.0000e+00  3e+03  1e+01  5e+01  1e+00
 1:  1.8366e+00  8.2813e-01  9e+02  6e+00  3e+01  5e-03
 2:  2.8763e+00  2.4966e+00  1e+02  2e+00  1e+01  1e-03
 3:  3.4712e+00  3.2012e+00  7e+01  2e+00  7e+00  1e-03
 4:  3.4783e+00  3.2150e+00  7e+01  2e+00  7e+00  1e-03
 5:  4.0737e+00  3.8694e+00  5e+01  1e+00  5e+00  1e-03
 6:  4.1269e+00  3.9191e+00  5e+01  1e+00  5e+00  1e-03
 7:  5.9089e+00  5.8107e+00  2e+01  6e-01  3e+00  6e-04
 8:  5.9105e+00  5.8088e+00  2e+01  6e-01  3e+00  8e-04
 9:  7.6219e+00  7.5472e+00  2e+01  5e-01  2e+00  2e-03
10:  9.0172e+00  8.9862e+00  8e+00  2e-01  8e-01  1e-03
11:  8.9520e+00  8.9139e+00  1e+01  3e-01  1e+00  1e-03
12:  9.1054e+00  9.0713e+00  9e+00  2e-01  9e-01  1e-03
13:  1.0135e+01  1.0129e+01  2e+00  4e-02  2e-01  2e-04
14:  1.0292e+01  1.0290e+01  6e-01  1e-02  6e-02  1e-04
15:  1.0386e+01  1.0386e+01  2e-02  6e-04  2e-03  5e-06
16:  1.0389e+01  1.0389e+01  2e-03  5e-05  2e-04  

In [539]:
m_bswap_ptm = process_matrix(m_bswap, plot=True, plot_name='bSWAP')
m_bswap_sdp_ptm = process_matrix(m_bswap_sdp, plot=True, plot_name='bSWAP SDP')

In [558]:
angle1_range = np.linspace(0,2*np.pi, 24)
angle2_range = np.linspace(0,2*np.pi, 24)
angle3_range = np.linspace(0,2*np.pi, 24)
fidelity = np.zeros((len(angle1_range), len(angle2_range), len(angle3_range)))
for angle1_id, angle1 in enumerate(angle1_range):
    for angle2_id, angle2 in enumerate(angle2_range):
        for angle3_id, angle3 in enumerate(angle3_range):
            rotation_matrix = np.identity(4, dtype=np.complex)
            rotation_matrix[1,1] = np.exp(1j*angle1)
            rotation_matrix[2,2] = np.exp(1j*angle2)
            rotation_matrix[3,3] = np.exp(1j*angle1)*np.exp(1j*angle2)
            
            rotation_matrix_1d = np.asarray([[1, 0, 0, -1j*np.exp(1j*angle3)], 
                                             [0, 1, 0, 0],
                                             [0, 0, 1, 0],
                                             [-1j*np.exp(-1j*angle3), 0, 0, 1]])/np.sqrt(2)
            rotation_matrix = rotation_matrix@rotation_matrix_1d@rotation_matrix_1d
        
        #rotation_matrix = np.reshape(np.transpose(np.reshape(rotation_matrix, (2,2,2,2)),(1,0,3,2)), (4,4))
        #print(rotation_matrix)
    
        #rotation_matrix=np.identity(4)
        #rotation_matrix_process = np.reshape(np.einsum('ij,kl->jkil', np.conj(rotation_matrix),rotation_matrix), (16,16))
            cphi_ideal_matrix = np.reshape(np.einsum('ij,kl->ikjl', np.conj(rotation_matrix),rotation_matrix), (16,16))
            cphi_ideal_matrix = process_matrix(cphi_ideal_matrix, plot=False, plot_name='bSWAP SDP expected')

            #print(angle1, angle2, (np.sum(cphi_ideal_matrix*m_cz_sdp_ptm)+4)/20)
            fidelity[angle1_id, angle2_id, angle3_id] = (np.sum(cphi_ideal_matrix*m_bswap_ptm))/16

angle_id =np.unravel_index(np.argmax(fidelity.ravel()), fidelity.shape)
angle1 = angle1_range[angle_id[0]]
angle2 = angle2_range[angle_id[1]]
angle3 = angle3_range[angle_id[2]]
print(angle1, angle2, angle3)
print(np.max(fidelity))

1.0927278795094932 0.2731819698773733 3.551365608405853
0.16815549975792546


In [512]:
pqjson.dump(type="bswap-tomography",name='20190508-2',params = bswap_tomography)

In [541]:
sqrt_bswap_pulse = pg.pmulti(390e-9, ('iq_ex_00-2-11', pg.rect, 0.2))

In [542]:
two_qubit_process_tomography.set_prepare_seq([sqrt_bswap_pulse])
sqrt_bswap_tomography = two_qubit_process_tomography.measure() 

+Z+Z+Z+Z
uncorreted: {'avg_cov0': -8404685497498, 'avg_cov1': -8639064361541, 'avg_cov2': -8177930807395, 'avg_cov3': -8928383624799}
corrected: {'avg_cov0': -8404685497498.0, 'avg_cov1': -8639064361541.0}
+Z+X+Z+Z
uncorreted: {'avg_cov0': -8423026929758, 'avg_cov1': -9003929746619, 'avg_cov2': -8188845202764, 'avg_cov3': -9293973757931}
corrected: {'avg_cov0': -8423026929758.0, 'avg_cov1': -9003929746619.0}
+Z+Y+Z+Z
uncorreted: {'avg_cov0': -8273467079382, 'avg_cov1': -9215408625726, 'avg_cov2': -8032490338792, 'avg_cov3': -9503318590549}
corrected: {'avg_cov0': -8273467079382.0, 'avg_cov1': -9215408625726.0}
+Z-Z+Z+Z
uncorreted: {'avg_cov0': -7801999401548, 'avg_cov1': -9925320801144, 'avg_cov2': -7543087095867, 'avg_cov3': -10199995842972}
corrected: {'avg_cov0': -7801999401548.0, 'avg_cov1': -9925320801144.0}
+Z-X+Z+Z
uncorreted: {'avg_cov0': -7830053488188, 'avg_cov1': -9733912888672, 'avg_cov2': -7576059610099, 'avg_cov3': -10008822699850}
corrected: {'avg_cov0': -7830053488188.0

+Z-X+Z+X
uncorreted: {'avg_cov0': -8547785183426, 'avg_cov1': -7515268750122, 'avg_cov2': -8351843798968, 'avg_cov3': -7807067485699}
corrected: {'avg_cov0': -8547785183426.0, 'avg_cov1': -7515268750122.0}
+Z-Y+Z+X
uncorreted: {'avg_cov0': -8159752266174, 'avg_cov1': -8617531055486, 'avg_cov2': -7935881578884, 'avg_cov3': -8899030980222}
corrected: {'avg_cov0': -8159752266174.0, 'avg_cov1': -8617531055486.0}
+X+Z+Z+X
uncorreted: {'avg_cov0': -8399714013133, 'avg_cov1': -9608569429597, 'avg_cov2': -8150278266265, 'avg_cov3': -9896983986169}
corrected: {'avg_cov0': -8399714013133.0, 'avg_cov1': -9608569429597.0}
+X+X+Z+X
uncorreted: {'avg_cov0': -8094773074306, 'avg_cov1': -10079276586136, 'avg_cov2': -7832229373395, 'avg_cov3': -10360750602337}
corrected: {'avg_cov0': -8094773074306.0, 'avg_cov1': -10079276586136.0}
+X+Y+Z+X
uncorreted: {'avg_cov0': -8342319113186, 'avg_cov1': -9211876997390, 'avg_cov2': -8100514156478, 'avg_cov3': -9500243724096}
corrected: {'avg_cov0': -8342319113186.

+X+Y+Z+Y
uncorreted: {'avg_cov0': -8154691376790, 'avg_cov1': -9742940266905, 'avg_cov2': -7901650840473, 'avg_cov3': -10026446425973}
corrected: {'avg_cov0': -8154691376790.0, 'avg_cov1': -9742940266905.0}
+X-Z+Z+Y
uncorreted: {'avg_cov0': -7880219792502, 'avg_cov1': -9751627756552, 'avg_cov2': -7625015235315, 'avg_cov3': -10026733920439}
corrected: {'avg_cov0': -7880219792502.0, 'avg_cov1': -9751627756552.0}
+X-X+Z+Y
uncorreted: {'avg_cov0': -8090376776178, 'avg_cov1': -9751039926035, 'avg_cov2': -7835813464179, 'avg_cov3': -10031984905634}
corrected: {'avg_cov0': -8090376776178.0, 'avg_cov1': -9751039926035.0}
+X-Y+Z+Y
uncorreted: {'avg_cov0': -8053391572380, 'avg_cov1': -9557612537437, 'avg_cov2': -7802019685386, 'avg_cov3': -9838886817308}
corrected: {'avg_cov0': -8053391572380.0, 'avg_cov1': -9557612537437.0}
+Y+Z+Z+Y
uncorreted: {'avg_cov0': -8793844641193, 'avg_cov1': -8326422107344, 'avg_cov2': -8576416200607, 'avg_cov3': -8628435262716}
corrected: {'avg_cov0': -8793844641193.

+Y+Z+Z-Z
uncorreted: {'avg_cov0': -8364185408802, 'avg_cov1': -9643400590424, 'avg_cov2': -8112530674857, 'avg_cov3': -9933112609242}
corrected: {'avg_cov0': -8364185408802.0, 'avg_cov1': -9643400590424.0}
+Y+X+Z-Z
uncorreted: {'avg_cov0': -8174473037039, 'avg_cov1': -9711788631837, 'avg_cov2': -7919833207929, 'avg_cov3': -9996399774972}
corrected: {'avg_cov0': -8174473037039.0, 'avg_cov1': -9711788631837.0}
+Y+Y+Z-Z
uncorreted: {'avg_cov0': -8638347433207, 'avg_cov1': -9422389498484, 'avg_cov2': -8393044921329, 'avg_cov3': -9719634654864}
corrected: {'avg_cov0': -8638347433207.0, 'avg_cov1': -9422389498484.0}
+Y-Z+Z-Z
uncorreted: {'avg_cov0': -8688164358452, 'avg_cov1': -8584368096670, 'avg_cov2': -8462635415546, 'avg_cov3': -8881501766197}
corrected: {'avg_cov0': -8688164358452.0, 'avg_cov1': -8584368096670.0}
+Y-X+Z-Z
uncorreted: {'avg_cov0': -8613697653543, 'avg_cov1': -8913437810944, 'avg_cov2': -8379447960988, 'avg_cov3': -9209353982244}
corrected: {'avg_cov0': -8613697653543.0, 

+Y-X+Z-X
uncorreted: {'avg_cov0': -8600739131206, 'avg_cov1': -8870936576743, 'avg_cov2': -8368856767440, 'avg_cov3': -9167303207607}
corrected: {'avg_cov0': -8600739131206.0, 'avg_cov1': -8870936576743.0}
+Y-Y+Z-X
uncorreted: {'avg_cov0': -8664048448464, 'avg_cov1': -8078597336975, 'avg_cov2': -8450738021194, 'avg_cov3': -8375170334370}
corrected: {'avg_cov0': -8664048448464.0, 'avg_cov1': -8078597336975.0}
-Z+Z+Z-X
uncorreted: {'avg_cov0': -8624400419382, 'avg_cov1': -9617192133626, 'avg_cov2': -8376441874020, 'avg_cov3': -9914060183902}
corrected: {'avg_cov0': -8624400419382.0, 'avg_cov1': -9617192133626.0}
-Z+X+Z-X
uncorreted: {'avg_cov0': -8271787071084, 'avg_cov1': -9925871681403, 'avg_cov2': -8013713999938, 'avg_cov3': -10211329600694}
corrected: {'avg_cov0': -8271787071084.0, 'avg_cov1': -9925871681403.0}
-Z+Y+Z-X
uncorreted: {'avg_cov0': -8230989393518, 'avg_cov1': -10538107173232, 'avg_cov2': -7956403368218, 'avg_cov3': -10823454356213}
corrected: {'avg_cov0': -8230989393518.

...

+Z-X-Y+X
uncorreted: {'avg_cov0': -8573567073226, 'avg_cov1': -7718769082906, 'avg_cov2': -8373095072942, 'avg_cov3': -8011305934289}
corrected: {'avg_cov0': -8573567073226.0, 'avg_cov1': -7718769082906.0}
+Z-Y-Y+X
uncorreted: {'avg_cov0': -8535137848105, 'avg_cov1': -9056601667200, 'avg_cov2': -8301103493525, 'avg_cov3': -9348799435628}
corrected: {'avg_cov0': -8535137848105.0, 'avg_cov1': -9056601667200.0}
+X+Z-Y+X
uncorreted: {'avg_cov0': -8160414297143, 'avg_cov1': -9467492933388, 'avg_cov2': -7913319116357, 'avg_cov3': -9751549352053}
corrected: {'avg_cov0': -8160414297143.0, 'avg_cov1': -9467492933388.0}
+X+X-Y+X
uncorreted: {'avg_cov0': -8430218461219, 'avg_cov1': -10281282217097, 'avg_cov2': -8163899468199, 'avg_cov3': -10574075639452}
corrected: {'avg_cov0': -8430218461219.0, 'avg_cov1': -10281282217097.0}
+X+Y-Y+X
uncorreted: {'avg_cov0': -8582142148624, 'avg_cov1': -8938407330303, 'avg_cov2': -8350363913805, 'avg_cov3': -9232594764246}
corrected: {'avg_cov0': -8582142148624.

+X+Y-Y+Y
uncorreted: {'avg_cov0': -8284963214038, 'avg_cov1': -9775731543907, 'avg_cov2': -8028994548011, 'avg_cov3': -10062494116724}
corrected: {'avg_cov0': -8284963214038.0, 'avg_cov1': -9775731543907.0}
+X-Z-Y+Y
uncorreted: {'avg_cov0': -8078505805613, 'avg_cov1': -9922776165699, 'avg_cov2': -7818669117738, 'avg_cov3': -10204000221977}
corrected: {'avg_cov0': -8078505805613.0, 'avg_cov1': -9922776165699.0}
+X-X-Y+Y
uncorreted: {'avg_cov0': -8222806567600, 'avg_cov1': -9516501587203, 'avg_cov2': -7974110056110, 'avg_cov3': -9801447249031}
corrected: {'avg_cov0': -8222806567600.0, 'avg_cov1': -9516501587203.0}
+X-Y-Y+Y
uncorreted: {'avg_cov0': -8479064573076, 'avg_cov1': -9939662183845, 'avg_cov2': -8220420917049, 'avg_cov3': -10231886619661}
corrected: {'avg_cov0': -8479064573076.0, 'avg_cov1': -9939662183845.0}
+Y+Z-Y+Y
uncorreted: {'avg_cov0': -9207020099432, 'avg_cov1': -8076813375567, 'avg_cov2': -8997551652516, 'avg_cov3': -8388544403717}
corrected: {'avg_cov0': -9207020099432.

+Y+Z-Y-Z
uncorreted: {'avg_cov0': -8488301435923, 'avg_cov1': -9809980063429, 'avg_cov2': -8232390864312, 'avg_cov3': -10102905367838}
corrected: {'avg_cov0': -8488301435923.0, 'avg_cov1': -9809980063429.0}
+Y+X-Y-Z
uncorreted: {'avg_cov0': -8409157115523, 'avg_cov1': -10185842042288, 'avg_cov2': -8143494958423, 'avg_cov3': -10477592590716}
corrected: {'avg_cov0': -8409157115523.0, 'avg_cov1': -10185842042288.0}
+Y+Y-Y-Z
uncorreted: {'avg_cov0': -8389106665653, 'avg_cov1': -9445659305092, 'avg_cov2': -8140423038442, 'avg_cov3': -9736746636256}
corrected: {'avg_cov0': -8389106665653.0, 'avg_cov1': -9445659305092.0}
+Y-Z-Y-Z
uncorreted: {'avg_cov0': -8424093486125, 'avg_cov1': -8189749447257, 'avg_cov2': -8207498787500, 'avg_cov3': -8479858437159}
corrected: {'avg_cov0': -8424093486125.0, 'avg_cov1': -8189749447257.0}
+Y-X-Y-Z
uncorreted: {'avg_cov0': -8441571961617, 'avg_cov1': -8737042035831, 'avg_cov2': -8212889405006, 'avg_cov3': -9028177072829}
corrected: {'avg_cov0': -8441571961617

+Y-X-Y-X
uncorreted: {'avg_cov0': -8361944330067, 'avg_cov1': -8676066530969, 'avg_cov2': -8133905911687, 'avg_cov3': -8964152943495}
corrected: {'avg_cov0': -8361944330067.0, 'avg_cov1': -8676066530969.0}
+Y-Y-Y-X
uncorreted: {'avg_cov0': -9022426236510, 'avg_cov1': -8313232739535, 'avg_cov2': -8805175009945, 'avg_cov3': -8619636285759}
corrected: {'avg_cov0': -9022426236510.0, 'avg_cov1': -8313232739535.0}
-Z+Z-Y-X
uncorreted: {'avg_cov0': -8462970171143, 'avg_cov1': -9457390366285, 'avg_cov2': -8215623505384, 'avg_cov3': -9750592753993}
corrected: {'avg_cov0': -8462970171143.0, 'avg_cov1': -9457390366285.0}
-Z+X-Y-X
uncorreted: {'avg_cov0': -8382709141478, 'avg_cov1': -10168920435383, 'avg_cov2': -8116065505656, 'avg_cov3': -10459891682440}
corrected: {'avg_cov0': -8382709141478.0, 'avg_cov1': -10168920435383.0}
-Z+Y-Y-X
uncorreted: {'avg_cov0': -8210564720453, 'avg_cov1': -10034730132369, 'avg_cov2': -7948087362030, 'avg_cov3': -10320042670665}
corrected: {'avg_cov0': -821056472045

-Z+Y-Y-Y
uncorreted: {'avg_cov0': -8293912608324, 'avg_cov1': -10115862273625, 'avg_cov2': -8031341120706, 'avg_cov3': -10402469954729}
corrected: {'avg_cov0': -8293912608324.0, 'avg_cov1': -10115862273625.0}
-Z-Z-Y-Y
uncorreted: {'avg_cov0': -8639244425686, 'avg_cov1': -9195873387360, 'avg_cov2': -8400811903255, 'avg_cov3': -9491781655392}
corrected: {'avg_cov0': -8639244425686.0, 'avg_cov1': -9195873387360.0}
-Z-X-Y-Y
uncorreted: {'avg_cov0': -7938121872828, 'avg_cov1': -10228888451322, 'avg_cov2': -7672027772196, 'avg_cov3': -10507849425988}
corrected: {'avg_cov0': -7938121872828.0, 'avg_cov1': -10228888451322.0}
-Z-Y-Y-Y
uncorreted: {'avg_cov0': -8489277797354, 'avg_cov1': -9650425157592, 'avg_cov2': -8239275023947, 'avg_cov3': -9941895346417}
corrected: {'avg_cov0': -8489277797354.0, 'avg_cov1': -9650425157592.0}
-X+Z-Y-Y
uncorreted: {'avg_cov0': -7947465731018, 'avg_cov1': -10499315404022, 'avg_cov2': -7674621122631, 'avg_cov3': -10775818850038}
corrected: {'avg_cov0': -794746573

In [543]:
pqjson.dump(type="sqrt-bswap-tomography",name='20190508-2',params = sqrt_bswap_tomography)

In [545]:
m0_sqrt_bswap = np.zeros((36,36))
m1_sqrt_bswap = np.zeros((36,36))
for left_state_id, left_state in enumerate(cube_state_prod):
    for right_state_id, right_state in enumerate(cube_state_prod):
        m0_sqrt_bswap[left_state_id, right_state_id] = sqrt_bswap_tomography[left_state+right_state+'-Pavg_cov0']
        m1_sqrt_bswap[left_state_id, right_state_id] = sqrt_bswap_tomography[left_state+right_state+'-Pavg_cov1']

figs, axes = plt.subplots(1,2, figsize=(12,5))
axes[0].pcolormesh(m0_sqrt_bswap)
axes[0].set_title('avg_cov0')
axes[0].grid(True)
axes[0].set_xticks(np.arange(36))
axes[0].set_yticks(np.arange(36)[::-1])
axes[1].pcolormesh(m1_sqrt_bswap)
axes[1].set_title('avg_cov1')
axes[1].grid(True)
axes[1].set_xticks(np.arange(36))
axes[1].set_yticks(np.arange(36)[::-1])
fig = plt.gcf()
fig.canvas.set_window_title('Raw measurements SQRT bSWAP measured')

In [546]:
m_sqrt_bswap = np.zeros((16,16), dtype=np.complex)
for left_state_id, left_state in enumerate(np.arange(16)):
    for right_state_id, right_state in enumerate(np.arange(16)):
        m_sqrt_bswap[left_state_id, right_state_id] = sqrt_bswap_tomography[str(left_state)+','+str(right_state)]

In [548]:
fig, axes = plt.subplots(1,2, figsize=(8,4))
fig = plt.gcf()
fig.canvas.set_window_title('Chi-matrix measured')
axes[0].pcolormesh(np.rot90(np.real(m_sqrt_bswap)),cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_xticks(np.arange(16))
axes[0].set_yticks(np.arange(16)[::-1])
axes[0].grid(True)

axes[1].pcolormesh(np.rot90(np.imag(m_sqrt_bswap)),cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_xticks(np.arange(16))
axes[1].set_yticks(np.arange(16)[::-1])
axes[1].grid(True)

In [550]:
m_sqrt_bswap_sdp = process_tomography_measurement_results_to_chi_matrix(sqrt_bswap_tomography)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.0000e+00  3e+03  1e+01  5e+01  1e+00
 1:  2.5507e+00  1.5395e+00  9e+02  6e+00  3e+01  5e-03
 2:  4.0967e+00  3.7070e+00  1e+02  2e+00  1e+01  2e-03
 3:  5.0620e+00  4.7825e+00  8e+01  2e+00  7e+00  2e-03
 4:  5.5660e+00  5.3137e+00  6e+01  1e+00  7e+00  3e-03
 5:  6.9228e+00  6.7510e+00  5e+01  1e+00  4e+00  2e-03
 6:  6.9481e+00  6.7703e+00  5e+01  1e+00  5e+00  3e-03
 7:  9.7918e+00  9.7133e+00  2e+01  5e-01  2e+00  1e-03
 8:  9.7638e+00  9.6811e+00  2e+01  5e-01  2e+00  1e-03
 9:  1.2667e+01  1.2636e+01  1e+01  2e-01  8e-01  1e-03
10:  1.2622e+01  1.2589e+01  1e+01  2e-01  9e-01  1e-03
11:  1.3844e+01  1.3835e+01  3e+00  6e-02  3e-01  5e-04
12:  1.3843e+01  1.3833e+01  3e+00  6e-02  3e-01  5e-04
13:  1.4275e+01  1.4274e+01  7e-01  1e-02  5e-02  1e-04
14:  1.4346e+01  1.4345e+01  2e-01  4e-03  2e-02  5e-05
15:  1.4384e+01  1.4384e+01  2e-02  5e-04  2e-03  5e-06
16:  1.4388e+01  1.4388e+01  3e-03  5e-05  2e-04  

In [551]:
m_sqrt_bswap_ptm = process_matrix(m_sqrt_bswap, plot=True, plot_name='bSWAP')
m_sqrt_bswap_sdp_ptm = process_matrix(m_sqrt_bswap_sdp, plot=True, plot_name='bSWAP SDP')

In [555]:
angle1_range = np.linspace(0,2*np.pi, 24)
angle2_range = np.linspace(0,2*np.pi, 24)
angle3_range = np.linspace(0,2*np.pi, 24)
fidelity = np.zeros((len(angle1_range), len(angle2_range), len(angle3_range)))
for angle1_id, angle1 in enumerate(angle1_range):
    for angle2_id, angle2 in enumerate(angle2_range):
        for angle3_id, angle3 in enumerate(angle3_range):
            rotation_matrix = np.identity(4, dtype=np.complex)
            rotation_matrix[1,1] = np.exp(1j*angle1)
            rotation_matrix[2,2] = np.exp(1j*angle2)
            rotation_matrix[3,3] = np.exp(1j*angle1)*np.exp(1j*angle2)
            
            rotation_matrix_1d = np.asarray([[1, 0, 0, -1j*np.exp(1j*angle3)], 
                                             [0, 1, 0, 0],
                                             [0, 0, 1, 0],
                                             [-1j*np.exp(-1j*angle3), 0, 0, 1]])/np.sqrt(2)
            rotation_matrix = rotation_matrix@rotation_matrix_1d
        
        #rotation_matrix = np.reshape(np.transpose(np.reshape(rotation_matrix, (2,2,2,2)),(1,0,3,2)), (4,4))
        #print(rotation_matrix)
    
        #rotation_matrix=np.identity(4)
        #rotation_matrix_process = np.reshape(np.einsum('ij,kl->jkil', np.conj(rotation_matrix),rotation_matrix), (16,16))
            cphi_ideal_matrix = np.reshape(np.einsum('ij,kl->ikjl', np.conj(rotation_matrix),rotation_matrix), (16,16))
            cphi_ideal_matrix = process_matrix(cphi_ideal_matrix, plot=False, plot_name='bSWAP SDP expected')

            #print(angle1, angle2, (np.sum(cphi_ideal_matrix*m_cz_sdp_ptm)+4)/20)
            fidelity[angle1_id, angle2_id, angle3_id] = (np.sum(cphi_ideal_matrix*m_sqrt_bswap_ptm))/16

angle_id =np.unravel_index(np.argmax(fidelity.ravel()), fidelity.shape)
angle1 = angle1_range[angle_id[0]]
angle2 = angle2_range[angle_id[1]]
angle3 = angle3_range[angle_id[2]]
print(angle1, angle2, angle3)
print(np.max(fidelity))

5.463639397547466 1.9122737891416133 3.2781836385284797
0.2290976081538924
